# Sistema experto de generación de datos para entrenamiento de GNN


---



### Entorno.py

In [ ]:
'''En este modulo se encuentra la funcion de generación de escenarios
en el cual se generan los escenarios de entrada para el algoritmo centralizado/distribuido, estos serán las posiciones
iniciales de todos los drones, objetivos y estación de carga (se asume que todos los drones empiezan con 100% de batería y desde la estación de carga
, en la misma posición por conveniencia)'''
import numpy as np
import math
from random import randrange
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython import display
# Funciones de comportamiento de la bateria
'''
Versión 1 usando la función lineal de descarga de la batería (fuente: dji)

def descarga_bateria(estado_actual):
    t = (100 - estado_actual)*(55/100)
    return round(-(100/55)*(t+1) + 100)

def carga_bateria(estado_actual):
    return round(estado_actual+(100/30))

    '''

'''Versión 2 usando una cadena de Markov'''


def descarga_bateria(estado_actual):
    if(random.random()>0.85 and estado_actual>0):
        return estado_actual-1
    else:
        return estado_actual

def carga_bateria(estado_actual):
    if(random.random()>0.05 and estado_actual<10):
        return estado_actual+1
    else:
        return estado_actual


#dim dimensión nxn de la grid, las estaciónes de carga estarán en las posiciónes (0,0) y (15,15)

def estado_inicio(dim, n_robots, n_obj):
    estado = {}
    pos_robots = []
    for i in range(0, n_robots):
        if(random.random()>0.5):
            pos_robots.append([0, randrange(0, dim-1)])
        else:
            pos_robots.append([randrange(0, dim-1), 0])
    pos_robots = np.array(pos_robots, dtype='float32')
    #bateria = np.ones(n_robots)*100
    bateria = np.ones(n_robots)*10
    pos_obj = []

    for i in range(0, n_obj):
        pos_obj.append([randrange(1, dim-1), randrange(1, dim-1)])
    pos_obj = np.array(pos_obj, dtype='float32')

    estado['pos_robots'] = pos_robots
    estado['bateria'] = bateria
    estado['pos_obj'] = pos_obj

    return estado


class Estado:

    # constructor
    def __init__(self, pos_robots, bateria, pos_obj):
        self.pos_robots = pos_robots
        self.bateria = bateria
        self.pos_obj = pos_obj
        self.registro = []
        self.it = 0
        self.fov_range = 4
        self.obj_ya_cubierto = np.ones(len(pos_robots))*-1

    def reset_env(self, pos_robots, bateria, pos_obj):
        self.pos_robots = pos_robots
        self.bateria = bateria
        self.pos_obj = pos_obj

    def get_estado(self):
        estado = {}
        estado = (map(tuple,self.pos_robots.tolist()), tuple(self.bateria), map(tuple, self.pos_obj.tolist()), tuple(self.planificador()))
        estado = tuple(map(tuple, estado))

        return estado

    def get_estado_array(self):
        #return np.array([np.concatenate([self.pos_robots.flatten(), self.bateria.flatten(), self.pos_obj.flatten(), self.planificador().flatten()])])
        # claro porque el no sabe donde estan los obj, se guia por la info que obtiene del FOV de cada robot (usando el planificador)
        return np.array([np.concatenate([self.pos_robots.flatten(), self.bateria.flatten(), self.planificador().flatten()])])

    def get_estado_experto(self):
        return [self.pos_robots, self.bateria.flatten(), self.planificador().flatten(), self.obj_ya_cubierto]


    def planificador(self):
      #planificacion = np.random.randint(0,6, size=len(self.pos_robots))
      planificacion = np.ones(len(self.pos_robots))*40 # un numero alejado de sus opciones lo interpretarán como fuera del alcance de cualquier objetivo
      direcciones = {
          (0,1): 0,
          (1,1): 1,
          (1,-1): 2,
          (0,-1): 3,
          (-1,-1): 4,
          (-1,1): 5,
          (1,0): 2,
          (-1,0): 4
      }
      for i in range(0, len(self.pos_robots)):
        for j in range(0, len(self.pos_obj)):
            if(math.sqrt((self.pos_robots[i,0]-self.pos_obj[j,0])**2 + (self.pos_robots[i,1]-self.pos_obj[j,1])**2)<=self.fov_range):
              dir = (self.pos_obj[j] - self.pos_robots[i])/abs(self.pos_obj[j] - self.pos_robots[i])
              x = np.nan_to_num(dir)
              if(tuple(x) in direcciones):
                planificacion[i] = direcciones[tuple(x)]
      return planificacion


    def verif_accion(self, robot, accion):
        nueva_pos = None
        acciones_contrarias = {0:3, 1:4, 2:5, 3:0, 4:1, 5:2}
        acciones_esquinas = {(0, 0): 1, (30, 0): 5, (30, 30): 4, (0, 30): 2}
        if(accion==0):
            nueva_pos = [self.pos_robots[robot,0], self.pos_robots[robot,1]+1]
        elif(accion==1):
            nueva_pos = [self.pos_robots[robot,0]+1, self.pos_robots[robot,1]+0.5]
        elif(accion==2):
            nueva_pos = [self.pos_robots[robot,0]+1, self.pos_robots[robot,1]-0.5]
        elif(accion==3):
            nueva_pos = [self.pos_robots[robot,0], self.pos_robots[robot,1]-1]
        elif(accion==4):
            nueva_pos = [self.pos_robots[robot,0]-1, self.pos_robots[robot,1]-0.5]
        elif(accion==5):
            nueva_pos = [self.pos_robots[robot,0]-1, self.pos_robots[robot,1]+0.5]

        if(nueva_pos == None):
          return accion
        elif(tuple(nueva_pos) in acciones_esquinas):
          return acciones_esquinas[tuple(nueva_pos)]
        elif(nueva_pos[0]>30 or nueva_pos[0]<0 or nueva_pos[1]>30 or nueva_pos[1]<0):
          return acciones_contrarias[accion]
        else:
          return accion



    def step(self, accion):
        # actualizar posicion de todos los i robots
        for i in range(0, len(self.pos_robots)):

            if(self.bateria[i]<=0):
                continue

            accion[i] = self.verif_accion(i, accion[i]) #correcion de acciones, en caso de que se intente salir del limite del mapa

            if(accion[i]==0):
                self.pos_robots[i] = [self.pos_robots[i,0], self.pos_robots[i,1]+1]
            elif(accion[i]==1):
                self.pos_robots[i] = [self.pos_robots[i,0]+1, self.pos_robots[i,1]+0.5]
            elif(accion[i]==2):
                self.pos_robots[i] = [self.pos_robots[i,0]+1, self.pos_robots[i,1]-0.5]
            elif(accion[i]==3):
                self.pos_robots[i] = [self.pos_robots[i,0], self.pos_robots[i,1]-1]
            elif(accion[i]==4):
                self.pos_robots[i] = [self.pos_robots[i,0]-1, self.pos_robots[i,1]-0.5]
            elif(accion[i]==5):
                self.pos_robots[i] = [self.pos_robots[i,0]-1, self.pos_robots[i,1]+0.5]
            else:
                pass # la accion 6 es que elige mantenerse en posicion

            # actualizar el estado de la bateria
            if((self.pos_robots[i]==[0,0]).all() or (self.pos_robots[i]==[15,15]).all()):
                self.bateria[i] = carga_bateria(self.bateria[i])
            else:
                self.bateria[i] = descarga_bateria(self.bateria[i])

            #actualizar pos objetivos
        for i in range(0, len(self.pos_obj)):
            dir = np.random.choice([0,1,2,3,4,5,6], p=[0, 0.15, 0.1, 0, 0.05, 0, 0.7])
            if(dir==0 and self.pos_obj[i,0]<30 and self.pos_obj[i,1]+1<30):
                self.pos_obj[i] = [self.pos_obj[i,0], self.pos_obj[i,1]+1]
            elif(dir==1 and self.pos_obj[i,0]+1<30 and self.pos_obj[i,1]+0.5<30):
                self.pos_obj[i] = [self.pos_obj[i,0]+1, self.pos_obj[i,1]+0.5]
            elif(dir==2 and self.pos_obj[i,0]+1<30 and self.pos_obj[i,1]-0.5>=0):
                self.pos_obj[i] = [self.pos_obj[i,0]+1, self.pos_obj[i,1]-0.5]
            elif(dir==3 and self.pos_obj[i,0]<30 and self.pos_obj[i,1]-1>=0):
                self.pos_obj[i] = [self.pos_obj[i,0], self.pos_obj[i,1]-1]
            elif(dir==4 and self.pos_obj[i,0]-1>=0 and self.pos_obj[i,1]-0.5>=0):
                self.pos_obj[i] = [self.pos_obj[i,0]-1, self.pos_obj[i,1]-0.5]
            elif(dir==5 and self.pos_obj[i,0]-1>=0 and self.pos_obj[i,1]+0.5<30):
                self.pos_obj[i] = [self.pos_obj[i,0]-1, self.pos_obj[i,1]+0.5]
            else:
                pass
        self.insertar_registro()

        #actualizamos el vector obj_ya_cubierto
        for i in range(0, len(self.pos_robots)):
            if(self.bateria[i]<=0):
                self.obj_ya_cubierto[i] = -1
                continue
            FOV1 = {}
            for k in [0, 0.5, 1]:
                for l in [0, 0.5, 1]:
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]] = 0
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]] = 0

            self.obj_ya_cubierto[i] = -1 # primero empezamos diciendo que no está cubriendo a nadie, y después comprobamos

            for j in range(0, len(self.pos_obj)):
                if(tuple(self.pos_obj[j]) in FOV1):
                    self.obj_ya_cubierto[i] = j



    def evaluar_posiciones(self):
        #para cada robot, se va a evaluar su posición utilizando el siguiente proceso:
        # recorriendo la matriz donde está la posición de cada robot:
            # para cada posición de robot (osea, para cada robot) generar el FOV (uno de radio 1)
             # con el de radio 1 verificar y contar la cantidad de obj cuyas posiciónes estan en él
             # meter en un vector en cada posición para cada robot la siguiente operación: 100*n_obj_FOV_1 - penalizacion
        #Retornar el vector resultante (que serán las recompensas del sistema en el estado actual tras evaluarse). Esto servirá para la funcion R(s,a) ya que esta es
        # literal la recompensa por el estado en el que se está
        vector_recompensa = np.zeros(len(self.pos_robots))

        obj_ya_cubierto_copia = self.obj_ya_cubierto.copy()

        for i in range(0, len(self.pos_robots)):
            if(self.bateria[i]<=0):
                continue
            penalizacion = 0
            FOV1 = {}
            for k in [0, 0.5, 1]:
                for l in [0, 0.5, 1]:
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]] = 0
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]] = 0

            obj_ya_cubierto_copia[i] = -1 # primero empezamos diciendo que no está cubriendo a nadie, y después comprobamos

            for j in range(0, len(self.pos_obj)):
                if(tuple(self.pos_obj[j]) in FOV1 and j not in obj_ya_cubierto_copia):
                    FOV1[tuple(self.pos_obj[j])] +=1
                    obj_ya_cubierto_copia[i] = j

            if((self.pos_robots[i]==[0,0]).all() or (self.pos_robots[i]==[15,15]).all()):
              penalizacion = -20
            else:
              penalizacion = 0

            vector_recompensa[i] = 100*sum(FOV1.values()) + penalizacion
        return sum(vector_recompensa)

    def insertar_registro(self):
      self.registro.append([self.it , self.pos_robots[:,0].copy(), self.pos_robots[:,1].copy(), self.pos_obj[:,0].copy(), self.pos_obj[:,1].copy()])
      self.it = self.it + 1

    def obtener_registro(self):
      return self.registro

    def generar_animacion(self):
      #obtenemos las ubicaciones de los robots y los objetivos
      robots = []
      for it in range(0, self.it):
        robots.append(pd.DataFrame([it, self.obtener_registro()[it][1], self.obtener_registro()[it][2]]).transpose())
      df_robots = pd.concat(robots)
      df_robots.columns = ['it', 'x0', 'y0']

      objetivos = []
      for it in range(0, self.it):
        objetivos.append(pd.DataFrame([self.obtener_registro()[it][0], self.obtener_registro()[it][3], self.obtener_registro()[it][4]]).transpose())
      df_objetivos = pd.concat(objetivos)
      df_objetivos.columns = ['it', 'x0', 'y0']

      fig, ax = plt.subplots()
      ax.set_xlim([0, 31])
      ax.set_ylim([0, 31])

      def animate(i):
        p1 = ax.clear()
        p2 = ax.clear()
        ax.grid(True)
        ax.set_xticks(np.linspace(0, 30, 31))
        ax.set_yticks(np.linspace(0, 30, 31))
        df_img1 = df_robots[df_robots['it'] == i][['x0', 'y0']]
        df_img2 = df_objetivos[df_objetivos['it'] == i][['x0', 'y0']]
        ax.set_xlim([0, 31])
        ax.set_ylim([0, 31])
        p1 = plt.scatter(df_img1['x0'].squeeze(), df_img1['y0'].squeeze())
        p2 = plt.scatter(df_img2['x0'].squeeze(), df_img2['y0'].squeeze())

        return p1,p2

      ani = animation.FuncAnimation(fig, func = animate, frames = self.it, interval = 400, blit = True)
      ani.save("animacion.gif", writer="Pillow")


### Solución Experta/Voraz (programada)

In [ ]:
# definición del algoritmo experto a partir de una lógica voraz (humana/programada)

'''estado=[baterias, LiDAR, objetivo_cubiertos]'''
def algoritmo_voraz(estado, n_robots):
    acciones_contrarias = {0:3, 1:4, 2:5, 3:0, 4:1, 5:2, 40:random.randint(0,5), 6:6}
    direcciones = {
        (0,1): 0,
        (1,1): 1,
        (1,-1): 2,
        (0,-1): 3,
        (-1,-1): 4,
        (-1,1): 5,
        (1,0): 2,
        (-1,0): 4,
        (0,0): 6
    }
    acciones = np.ones(n_robots)*6
    pos, baterias, LiDAR, objetivos_cubiertos = estado
    for i in range(0, n_robots):
        if(baterias[i]<=3):
            dir = ([15,15]-pos[i])/abs([15,15]-pos[i])
            x = np.nan_to_num(dir)
            acciones[i] = direcciones[tuple(x)] #dejar todo e ir a la estacion de carga
        else:
            if(objetivos_cubiertos[i] != -1 and objetivos_cubiertos[i] not in np.delete(objetivos_cubiertos, i)):
                acciones[i] = 6
            elif(objetivos_cubiertos[i] != -1 and objetivos_cubiertos[i] in np.delete(objetivos_cubiertos, i)):
                acciones[i] = acciones_contrarias[LiDAR[i]]
                objetivos_cubiertos[i] = -1
            elif(LiDAR[i] != 40):
                acciones[i] = LiDAR[i]
            else:
                acciones[i] = random.randint(0,5)

    return acciones


### Algoritmo Genetico


In [ ]:
import random
import numpy as np
import itertools
import copy

def algoritmo_genetico(estado, poblacion_ini, generaciones = 15):
  n_robots = len(estado.get_estado()[0])
  poblacion = []
  #conjunto de acciones
  for i in itertools.product([0,1,2,3,4,5,6], repeat=n_robots):
      poblacion.append(list(i))
  poblacion = random.sample(poblacion,int((poblacion_ini/100)*len(poblacion)))

  for generacion in range(generaciones):
    #evaluar el fitness
    fitness_candidatos = []
    for val in poblacion:
      candidato = copy.copy(val)
      estado_aux = copy.copy(estado)
      estado_aux.step(candidato)
      candidato.append(estado_aux.evaluar_posiciones())

      fitness_candidatos.append(candidato)

    fitness_candidatos = np.array(fitness_candidatos)

    if(sum(fitness_candidatos[:, -1]) == 0):
      # si resulta que no hay ninguna solucion viable en la poblacion inicial, que pare ahí y devuelva random
      return np.random.randint(low=0, high=6, size=n_robots)

    #selecionamos los mejores
    fitness_candidatos = fitness_candidatos[np.argsort(fitness_candidatos[:, -1])]
    n_seleccion = int(len(fitness_candidatos)*0.1)
    padre1, padre2 = fitness_candidatos[:n_seleccion], fitness_candidatos[n_seleccion:n_seleccion*2]
    padre1, padre2 = padre1[:, :-1], padre2[:, :-1] #quitamos la columna del fitness

    #generamos hijos

    hijos = []
    for _ in range(10):
      if(len(padre1[0])%2 !=0):
        mitad = len(padre1[0])//2
      else:
        mitad = len(padre1[0])//2
      hijo = np.concatenate((padre1[:,:mitad], padre2[:,mitad:]), axis=1)
      hijos.append(hijo)
      np.random.shuffle(padre1)
    hijos = np.vstack(hijos)

    #mutacion (adjacent swap de dos genes)
    gen1, gen2 = np.random.choice(hijos.shape[1], 2, replace=False)
    hijos_mutados = np.copy(hijos)
    hijos_mutados[:, [gen1, gen2]] = hijos[:, [gen2, gen1]]

    poblacion = hijos_mutados.tolist()

  fitness_candidatos = []
  for val in poblacion:
    candidato = copy.copy(val)
    estado_aux = copy.copy(estado)
    estado_aux.step(candidato)
    candidato.append(estado_aux.evaluar_posiciones())

    fitness_candidatos.append(candidato)

  fitness_candidatos = np.array(fitness_candidatos)

  return fitness_candidatos[0]





### DDPG_SistemaExperto.py

In [ ]:
'''Definición del sistema experto para generación de los datos de entrenamiento del sistema de inteligencia distribuida.
En él, se definirá el MDP el algoritmo de Deep-RL que lo resolverá (DDPG) y finalmente una fución para generar (en un txt, csv....)
el conjutno de datos de entramiento que serán simplemente el conjunto de estados explorados total y la matriz de acciones elegidas para el conjunto
según el estado. Este será preprocesado y transformado por el gnn_data_setup
'''
import math
import numpy as np
import random
import itertools
import tensorflow as tf

#Definimos las arquitecturas de las dos redes Actor-Critic

def create_actor_model(n_robots, n_obj):
    ini_pesos = tf.random_uniform_initializer(minval=-0.005, maxval=0.005)
    input = tf.keras.layers.Input(shape=(4*n_robots,))  #el segundo n_robots es por las baterías; los 2* es porque cada robot tiene 2 valores en coordenadas
    d1 = tf.keras.layers.Dense(252, activation="relu")(input)
    d2 = tf.keras.layers.Dense(128, activation="relu")(d1)
    d3 = tf.keras.layers.Dense(64, activation="relu")(d2)
    output_layer = tf.keras.layers.Dense(n_robots, activation="sigmoid", kernel_initializer=ini_pesos)(d3)
    #output_layer = tf.keras.layers.Dense(n_robots)(d4)

    output = output_layer*6
    #output = tf.keras.activations.relu(output_layer, max_value=6)

    return  tf.keras.Model(input, output)


def create_critic_model(n_robots, n_obj):
    # entrada del estado
    estado_input = tf.keras.Input(shape=(4*n_robots))
    estado_output = tf.keras.layers.Dense(64, activation = "relu")(estado_input)
    estado_output = tf.keras.layers.Dense(128, activation="relu")(estado_output)

    # entrada de las acciones
    acciones_input = tf.keras.Input(shape=(n_robots))
    acciones_output = tf.keras.layers.Dense(64, activation="relu")(acciones_input)
    acciones_output = tf.keras.layers.Dense(128, activation="relu")(acciones_output)

    inputs = tf.keras.layers.Concatenate()([estado_output, acciones_output])

    #resto de la acquitectura de la red
    d1 = tf.keras.layers.Dense(252, activation="relu")(inputs)
    d2 = tf.keras.layers.Dense(128, activation="relu")(d1)
    d3 = tf.keras.layers.Dense(64, activation="relu")(d2)
    output = tf.keras.layers.Dense(1)(d3)

    return tf.keras.Model([estado_input, acciones_input], output)

class TrainBuffer:
    def __init__(self, capacidad, batch_size):
        self.capacidad = capacidad
        self.batch = batch_size

        self.contador = 0

        self.buffer_estados = np.zeros((self.capacidad, 4*n_robots))
        self.buffer_acciones = np.zeros((self.capacidad, n_robots))
        self.buffer_recompensa = np.zeros((self.capacidad,1))
        self.buffer_sig_estado = np.zeros((self.capacidad, 4*n_robots))

    def record(self, obs):
        # cuando se sobrepasa la capacidad, se empiezan a sustituir los antiguos registros
        index = self.contador % self.capacidad

        self.buffer_estados[index] = obs[0]
        self.buffer_acciones[index] = obs[1]
        self.buffer_recompensa[index] = obs[2]
        self.buffer_sig_estado[index] = obs[3]

        self.contador += 1

    @tf.function
    def update(self, batch_estado, batch_acciones, batch_recompensa, batch_sig_estado):

        #entrenamiento de la red
        with tf.GradientTape() as tape:
            acciones_obj = actor_objetivo(batch_sig_estado, training=True)
            y = batch_recompensa + GAMMA*critic_objetivo([batch_sig_estado, acciones_obj], training=True)
            res_critico = critico([batch_estado, batch_acciones], training=True)
            critic_loss = tf.math.reduce_mean(tf.math.square(y - res_critico))

        grad_critic = tape.gradient(critic_loss, critico.trainable_variables)
        optimizador_critico.apply_gradients(zip(grad_critic, critico.trainable_variables))

        with tf.GradientTape() as tape:
            acciones = actor(batch_estado, training=True)
            res_critico = critico([batch_estado, acciones], training=True)

            actor_loss = -tf.math.reduce_mean(res_critico)

        grad_actor = tape.gradient(actor_loss, actor.trainable_variables)
        optimizador_actor.apply_gradients(zip(grad_actor, actor.trainable_variables))

    def learn(self):

        record_range = min(self.contador, self.capacidad)
        # se genera un batch aleatorio para el aprendizaje
        batch_indices = np.random.choice(record_range, self.batch)

        batch_estados = tf.convert_to_tensor(self.buffer_estados[batch_indices])
        batch_acciones = tf.convert_to_tensor(self.buffer_acciones[batch_indices])
        batch_recompensa = tf.convert_to_tensor(self.buffer_recompensa[batch_indices])
        batch_recompensa = tf.cast(batch_recompensa, dtype=tf.float32)
        batch_sig_estado = tf.convert_to_tensor(self.buffer_sig_estado[batch_indices])
        self.update(batch_estados, batch_acciones, batch_recompensa, batch_sig_estado)


#según el pseudocodigo
@tf.function
def update_target_weights(pesos_obj, pesos):
    for (a, b) in zip(pesos_obj, pesos):
        a.assign(b * TAU + a * (1 - TAU))

def policy3(estado, step):
  acciones = tf.squeeze(actor(estado))
  if(random.random() < math.pow(2, -step/40)):
    return np.random.rand(n_robots)
  num = np.random.randint(0, n_robots-1, size=2)
  rand_a = np.random.rand(2)
  acciones_proc = acciones.numpy()
  acciones_proc[num[0]], acciones_proc[num[1]] = rand_a[0], rand_a[1]

  return acciones_proc

def policy2(estado, step):
  acciones = tf.squeeze(actor(estado))
  # se agrega ruido mediante el adjacent swap de dos ordenes
  if(random.random() < math.pow(2, -step/40)):
    return np.random.rand(n_robots)
  num = random.randint(0,n_robots-2)
  acciones_proc = acciones.numpy()
  acciones_proc[num], acciones_proc[num+1] = acciones_proc[num+1], acciones_proc[num]

  return acciones_proc

def policy1(estado, step):
  #acciones = tf.squeeze(actor(normalizar_vec(estado)))
  acciones = tf.squeeze(actor(estado))
  if(random.random() < math.pow(2, -step/50)):
    return np.random.rand(n_robots)

  return acciones

def normalizar_vec(vec):
  vec_min = tf.math.reduce_min(vec)
  vec_max = tf.math.reduce_max(vec)
  return (vec-vec_min)/(vec_max-vec_min)

In [ ]:
''' Esto es lo que estará dentro del main() de DDGP_SistemaExperto.py'''
import numpy as np
from scipy import stats
import random
import itertools
import tensorflow as tf
import matplotlib.pyplot as plt

GAMMA = 0.8
SIMULACIONES = 60000
DIMMENSION = 30 # nxn
TAU = 0.005
n_robots = 5
n_obj = 6

estado_inicial = estado_inicio(DIMMENSION, n_robots, n_obj)
estado = Estado(estado_inicial['pos_robots'].copy(), estado_inicial['bateria'].copy(), estado_inicial['pos_obj'].copy())

#Declaramos los modelos con la misma arquitectura
actor = create_actor_model(n_robots, n_obj)
critico = create_critic_model(n_robots, n_obj)

actor_objetivo = create_actor_model(n_robots, n_obj)
critic_objetivo = create_critic_model(n_robots, n_obj)

# le ponemos los mismos pesos
actor_objetivo.set_weights(actor.get_weights())
critic_objetivo.set_weights(critico.get_weights())

optimizador_actor = tf.keras.optimizers.Adam(0.0001)
optimizador_critico = tf.keras.optimizers.Adam(0.0002)


buffer = TrainBuffer(100000, 128)
recompensas = []
simulaciones = []
#Train Loop

for j in range(SIMULACIONES):
  estado_inicial = estado_inicio(DIMMENSION, n_robots, n_obj)
  estado = Estado(estado_inicial['pos_robots'].copy(), estado_inicial['bateria'].copy(), estado_inicial['pos_obj'].copy())
  print("Estado inicial: ", estado.get_estado_array())
  estado0 = estado.get_estado_array()
  recompensa_por_simulacion = 0
  acciones = []
  for i in range(50):

      accion = policy1(estado0, i)
      accion_proc = np.round(accion)
      estado.step(accion_proc)
      estado1 = estado.get_estado_array()
      recompensa = estado.evaluar_posiciones()
      accion = np.array([accion])
      buffer.record((estado0, accion, recompensa, estado1))
      buffer.learn()
      estado0 = estado1.copy()
      acciones.append(accion_proc)

      update_target_weights(actor_objetivo.variables, actor.variables)
      update_target_weights(critic_objetivo.variables, critico.variables)
      recompensa_por_simulacion += recompensa
  print("Estado final: ", estado1)
  print("Mean acciones:", np.mean(acciones, axis=0))
  print("Mode acciones:", stats.mode(acciones)[0], " count array", stats.mode(acciones)[1])
  #print("Simulacion", j)
  #print("Pesos actor: ", actor.variables)
  #print("Pesos actor objetivo: ", actor_objetivo.variables)

  recompensas.append(recompensa_por_simulacion)
  simulaciones.append(j)
  print("Simulación ", j, " recompensa: ", recompensa_por_simulacion)


#Guardar los pesos
actor.save_weights("actor.h5")
actor_objetivo.save_weights("actor_objetivo.h5")

critico.save_weights("critico.h5")
critic_objetivo.save_weights("critico_objetivo.h5")

plt.plot(recompensas)
plt.show()

In [ ]:
''' Entrenamiento a apartir de una red preentrenada'''
import numpy as np
from scipy import stats
import random
import itertools
import tensorflow as tf
import matplotlib.pyplot as plt

GAMMA = 0.8
SIMULACIONES = 30000
DIMMENSION = 30 # nxn
TAU = 0.005
n_robots = 5
n_obj = 6

estado_inicial = estado_inicio(DIMMENSION, n_robots, n_obj)
estado = Estado(estado_inicial['pos_robots'].copy(), estado_inicial['bateria'].copy(), estado_inicial['pos_obj'].copy())

#Cargando solo los pesos a un modelo
actor = create_actor_model(n_robots, n_obj)
critico = create_critic_model(n_robots, n_obj)
actor_objetivo = create_actor_model(n_robots, n_obj)
critic_objetivo = create_critic_model(n_robots, n_obj)

actor.load_weights("./parametros/DDPG/4_robots_60000train/actor.h5")
critico.load_weights("./parametros/DDPG/4_robots_60000train/critico.h5")
actor_objetivo.load_weights("./parametros/DDPG/4_robots_60000train/actor_objetivo.h5")
critic_objetivo.load_weights("./parametros/DDPG/4_robots_60000train/critico_objetivo.h5")

#Cargar los modelos preentrenados enteros
#actor = tf.keras.models.load_model("parametros/DDPG/4_robots_final_train/modelo_actor")
#actor_objetivo = tf.keras.models.load_model("parametros/DDPG/4_robots_final_train/modelo_actor_objetivo")
#critico = tf.keras.models.load_model("parametros/DDPG/4_robots_final_train/modelo_critico")
#critic_objetivo = tf.keras.models.load_model("parametros/DDPG/4_robots_final_train/modelo_critico_objetivo")

optimizador_actor = tf.keras.optimizers.Adam(0.0001)
optimizador_critico = tf.keras.optimizers.Adam(0.0002)

buffer = TrainBuffer(100000, 128)
recompensas = []
simulaciones = []
#Train Loop

for j in range(SIMULACIONES):
  estado_inicial = estado_inicio(DIMMENSION, n_robots, n_obj)
  estado = Estado(estado_inicial['pos_robots'].copy(), estado_inicial['bateria'].copy(), estado_inicial['pos_obj'].copy())
  print("Estado inicial: ", estado.get_estado_array())
  estado0 = estado.get_estado_array()
  recompensa_por_simulacion = 0
  acciones = []
  for i in range(50):

      accion = policy1(estado0, i)
      accion_proc = np.round(accion)
      estado.step(accion_proc)
      estado1 = estado.get_estado_array()
      recompensa = estado.evaluar_posiciones()
      accion = np.array([accion])
      buffer.record((estado0, accion, recompensa, estado1))
      buffer.learn()
      estado0 = estado1.copy()
      acciones.append(accion_proc)

      update_target_weights(actor_objetivo.variables, actor.variables)
      update_target_weights(critic_objetivo.variables, critico.variables)
      recompensa_por_simulacion += recompensa
  print("Estado final: ", estado1)
  print("Mean acciones:", np.mean(acciones, axis=0))
  print("Mode acciones:", stats.mode(acciones)[0], " count array", stats.mode(acciones)[1])
  #print("Simulacion", j)
  #print("Pesos actor: ", actor.variables)
  #print("Pesos actor objetivo: ", actor_objetivo.variables)

  recompensas.append(recompensa_por_simulacion)
  simulaciones.append(j)
  print("Simulación ", j, " recompensa: ", recompensa_por_simulacion)

actor.save("parametros/DDPG/4_robots_final_train/modelo_actor")
actor_objetivo.save("parametros/DDPG/4_robots_final_train/modelo_actor_objetivo")

critico.save("parametros/DDPG/4_robots_final_train/modelo_critico")
critic_objetivo.save("parametros/DDPG/4_robots_final_train/modelo_critico_objetivo")

plt.plot(recompensas)
plt.show()

In [ ]:
'''Este es el generador del dataset de entrenamiento para la GNN a partir de los parametro de la red del Sitema experto calculados arriba'''

import pickle

SIMULACIONES = 100

DIMMENSION = 30 # nxn
n_robots = 5
n_obj = 6
steps = 50

def simulacion_DDPG(estado_inicial, DDPG_model, steps):
    recompensa_total=0
    df_atributos = []
    df_adj_mat = []
    df_acciones = []

    estado = Estado(estado_inicial['pos_robots'].copy(), estado_inicial['bateria'].copy(), estado_inicial['pos_obj'].copy())

    for j in range(steps):
        estado0 = estado.get_estado_array()
        datos_estado0 = estado.get_estado()
        accion = tf.squeeze(DDPG_model(estado0)).numpy()
        estado.step(np.round(accion))
        atributos, adj_mat, acciones = generar_datos(datos_estado0, np.round(accion))
        df_atributos.append(atributos)
        df_adj_mat.append(adj_mat)
        df_acciones.append(acciones)
        recompensa_total += estado.evaluar_posiciones()

    print(recompensa_total)
    return [np.array(df_adj_mat), df_atributos, np.array(df_acciones)]


#aqui inicia todo
DDPG_model = create_actor_model(n_robots, n_obj)
# cargamos los pesos del modelo
DDPG_model.load_weights("./parametros/DDPG/actor_objetivo.h5")

#DDPG_model = tf.keras.models.load_model("./parametros/4_robots_final_train/modelo_actor_objetivo")

for i in range(SIMULACIONES):
    estado_inicial = estado_inicio(DIMMENSION, n_robots, n_obj)
    #print(simulacion_DDPG(estado_inicial, DDPG_model))
    pickle.dump(simulacion_DDPG(estado_inicial, DDPG_model, steps), open(f'./data/DDPG/data_{i+1}.pkl', 'wb'))


In [ ]:
'''Este es el generador del dataset de entrenamiento para la GNN a partir de la Solucion Experta/Voraz'''

import pickle

SIMULACIONES = 100

DIMMENSION = 30 # nxn
n_robots = 5
n_obj = 6
steps = 50

def simulacion_Experta(estado_inicial, steps):
    recompensa_total=0
    df_atributos = []
    df_adj_mat = []
    df_acciones = []

    estado = Estado(estado_inicial['pos_robots'].copy(), estado_inicial['bateria'].copy(), estado_inicial['pos_obj'].copy())

    for j in range(steps):
        pos, baterias, LiDAR, objetivos_cubiertos = estado.get_estado_experto()
        datos_estado0 = estado.get_estado()
        accion = algoritmo_voraz([pos.copy(), baterias.copy(), LiDAR.copy(), objetivos_cubiertos.copy()], n_robots)
        estado.step(accion)
        atributos, adj_mat, acciones = generar_datos(datos_estado0, accion)
        df_atributos.append(atributos)
        df_adj_mat.append(adj_mat)
        df_acciones.append(acciones)
        recompensa_total += estado.evaluar_posiciones()
        #print(datos_estado0)

    print(recompensa_total)
    return [np.array(df_adj_mat), df_atributos, np.array(df_acciones)]

for i in range(SIMULACIONES):
    estado_inicial = estado_inicio(DIMMENSION, n_robots, n_obj)
    pickle.dump(simulacion_Experta(estado_inicial, steps), open(f'./data/Voraz/data_{i+1}.pkl', 'wb'))


### gnn_data_setup.py


---


In [ ]:
'''Aquí se definirán las funciones/clases para el preprocesado y transformación de los datos generados por el
Sistema Experto y crear el conjunto de entrenamiento'''
import numpy as np
import math
FOV_RANGE = 4
COM_RANGE = 6

def generar_datos(estado, acciones):
    adj_mat = generar_adj_mat(estado[0], COM_RANGE) # calculada a partir de pos_robots (valor de la distancia incluido)
    FOV_mat, FOV_mat_pos = FOV_obj(estado[0], estado[2], estado[1], FOV_RANGE)
    atributos = generar_atributos_v2(adj_mat, FOV_mat, estado[1], estado[0], estado[3]) # lista que contiene la posicion (o distancia) y el numero de objetivos ¿y sus posiciones? de cada robot

    acciones = np.array(acciones)

    return atributos, adj_mat, acciones

def generar_adj_mat(pos_robots, com_range):
    pos = np.array(pos_robots)
    adj_mat = np.zeros((len(pos), len(pos)))

    for i in range(0, len(pos)):
        for j in range(0, len(pos)):
            #if(np.linalg.norm(pos[i] - pos[j]) == 0 and i!=j):
             #   adj_mat[i,j] = 1
            if(math.sqrt((pos[i,0]-pos[j,0])**2 + (pos[i,1]-pos[j,1])**2) <= com_range and i!=j):
                #adj_mat[i,j] = math.sqrt((pos[i,0]-pos[j,0])**2 + (pos[i,1]-pos[j,1])**2)
                adj_mat[i,j] = 1
            else:
                adj_mat[i,j] = 0

    return adj_mat



def generar_atributos(adj_mat, fov_obj, fov_vec_pos, bateria, pos_robots, planificador):
    # matriz de num_robots x (1 + 1 + 1 + num_robots)
    # el primer atributo es igual para todos, el estado de batería
    # el segundo 1 es el número de objetivos cubiertos
    # el tercer 1 es el número de objetivos en el FOV del robot (cubiertos o no)
    # el conjutno de columnas num_robots almacena para cada robot la distancia relativa entre cada robot en su FOV (es)
    bateria = np.array(bateria)
    #atributos = np.zeros((len(adj_mat), (len(adj_mat)+2))) version anterior del vector de atributos, ahora sólo se deja un numero con el numero de vecinos
    atributos = np.zeros((len(adj_mat), 13))
    #atributos = {}
    # para cada robot i
    for i in range(0, len(adj_mat)):

        atributos[i, 0] = bateria[i]

        atributos[i, 1] = (fov_obj[i,:] == 1).sum() # numero de objetivos cubiertos
        atributos[i, 2] = (fov_obj[i,:] >= 1).sum() # numero de objetivos en su FOV cubiertos o no
        atributos[i, 3] = pos_robots[i][0] #pos del robot
        atributos[i, 4] = pos_robots[i][1] #pos del robot
        min_pos = sorted(fov_vec_pos[i])[:3]
        for idx, val in enumerate(min_pos):
          atributos[i, idx*2+5] = val[0]
          atributos[i, idx*2+6] = val[1]
        #it = 0
        #for j in range(0, len(adj_mat)-1):
         #   if(it!=i):
                #agregar el vector de distancia relativa entre el dron y el resto dentro de su FOV
          #      atributos[i, j+3] = adj_mat[i,it]
           # elif(it==i):
            #    it +=1
             #   atributos[i, j+3] = adj_mat[i,it]
            #it +=1
        if(pos_robots[i] == [0,0] or pos_robots[i] == [15,15]):
            atributos[i, 11] = 1 #cargando
        atributos[i, 12] = planificador[i]

    return atributos

def generar_atributos_v2(adj_mat, fov_obj, bateria, pos_robots, planificador):
    # esta versión de la función generar_atributos es igual que la anterior pero se eliminan los atributos del conjunto de columnas que representaban vectores de distancia relativa calculada a cada vecino
    bateria = np.array(bateria)
    atributos = np.zeros((len(adj_mat), 7))

    for i in range(0, len(adj_mat)):

        atributos[i, 0] = bateria[i]

        atributos[i, 1] = (fov_obj[i,:] == 1).sum() # numero de objetivos cubiertos
        atributos[i, 2] = (fov_obj[i,:] >= 1).sum() # numero de objetivos en su FOV cubiertos o no
        atributos[i, 3] = pos_robots[i][0] #pos del robot
        atributos[i, 4] = pos_robots[i][1] #pos del robot

        if(pos_robots[i] == [0,0] or pos_robots[i] == [15,15]):
            atributos[i, 5] = 1 #cargando
        atributos[i, 6] = planificador[i]

    return atributos

def FOV_obj(pos_robots, pos_obj, bateria, FOV_range):
    robots = np.array(pos_robots)
    obj = np.array(pos_obj)
    FOV_mat = np.zeros((len(robots), len(obj))) # num_robot x num_obj


    for i in range(0, len(pos_robots)):

      if(bateria[i] == 0):
        continue

      FOV1 = {}
      for m in [0, 0.5, 1]:
        for l in [0, 0.5, 1]:
            FOV1[pos_robots[i][0], pos_robots[i][1]] = 0
            FOV1[pos_robots[i][0]+m, pos_robots[i][1]] = 0
            FOV1[pos_robots[i][0]+m, pos_robots[i][1]+l] = 0
            FOV1[pos_robots[i][0], pos_robots[i][1]+l] = 0
            FOV1[pos_robots[i][0]-m, pos_robots[i][1]+l] = 0
            FOV1[pos_robots[i][0]-m, pos_robots[i][1]] = 0
            FOV1[pos_robots[i][0]-m, pos_robots[i][1]-l] = 0
            FOV1[pos_robots[i][0], pos_robots[i][1]-l] = 0
            FOV1[pos_robots[i][0]+m, pos_robots[i][1]-l] = 0

      for j in range(0, len(pos_obj)):
          if(tuple(pos_obj[j]) in FOV1):
              FOV_mat[i,j] = 1 # 1 si el robot i tiene al obj j cubierto


    for i in range(0, len(pos_robots)):

      if(bateria[i] == 0):
        continue

      FOV2 = {}
      for k in [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]:
        for l in [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]:
            FOV2[pos_robots[i][0]+k, pos_robots[i][1]] = 0
            FOV2[pos_robots[i][0]+k, pos_robots[i][1]+l] = 0
            FOV2[pos_robots[i][0], pos_robots[i][1]+l] = 0
            FOV2[pos_robots[i][0]-k, pos_robots[i][1]+l] = 0
            FOV2[pos_robots[i][0]-k, pos_robots[i][1]] = 0
            FOV2[pos_robots[i][0]-k, pos_robots[i][1]-l] = 0
            FOV2[pos_robots[i][0], pos_robots[i][1]-l] = 0
            FOV2[pos_robots[i][0]+k, pos_robots[i][1]-l] = 0

      for j in range(0, len(pos_obj)):
        if(tuple(pos_obj[j]) in FOV2 and FOV_mat[i,j] == 1):
          FOV_mat[i,j] = 1
        elif(tuple(pos_obj[j]) in FOV2 and FOV_mat[i,j] != 1):
          FOV_mat[i,j] += 2 # 2 si el robot i tiene al obj j cubierto o en su fov

    FOV_mat_pos = {}
    for i in range(0, len(pos_robots)):
      lista = [[0,0]]
      FOV2 = {}
      FOV_mat_pos[i] = lista
      if(bateria[i] == 0):
        continue

      for k in [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]:
        for l in [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]:
            FOV2[pos_robots[i][0]+k, pos_robots[i][1]] = 0
            FOV2[pos_robots[i][0]+k, pos_robots[i][1]+l] = 0
            FOV2[pos_robots[i][0], pos_robots[i][1]+l] = 0
            FOV2[pos_robots[i][0]-k, pos_robots[i][1]+l] = 0
            FOV2[pos_robots[i][0]-k, pos_robots[i][1]] = 0
            FOV2[pos_robots[i][0]-k, pos_robots[i][1]-l] = 0
            FOV2[pos_robots[i][0], pos_robots[i][1]-l] = 0
            FOV2[pos_robots[i][0]+k, pos_robots[i][1]-l] = 0

      #for j in range(0, len(pos_obj)):
       # if(tuple(pos_obj[j]) in FOV2):
        #  lista.append((np.array(pos_obj[j])-np.array(pos_robots[i])).tolist())
      #FOV_mat_pos[i] = lista
      for j in range(0, len(pos_robots)):
        if(i!=j and tuple(pos_robots[j]) in FOV2):
          lista.append((np.array(pos_robots[j])-np.array(pos_robots[i])).tolist())
      FOV_mat_pos[i] = lista

    return FOV_mat, FOV_mat_pos



# The GNN Model

CONSTRUCCIÓN DEL MODELO: en la experimentación se empezó utilizando simples operaciónes de **message passing** que es la que se utilizó en un articulo parecido al cual se le hace referencia en la memória de este proyecto. El message passing consiste en actualizar el vector de atributos del cada nodo mediante la agregación y transformación lineal de los vectores de atributos de sus vecinos con el propio del nodo incluido. <br>
En el articulo también aplicaron un paso anterior a la GNN de preprocesado que consistía en pasar el vector de atributos de cada vecino antes de agregarlo en la GNN por una mini MLP que preprocesara estos vectores, este paso también se aplica en el ejemplo del mutag, en el cual lo tiene más facil para aplicarlo ya que usan una función de tfgnn llamada MapFeatures que es aplicar una función map en el vector de atributos de cada nodo del grafo en la cual se puede incluir cualquier función de tranformación en este caso una capa Densa, el uso de esta función está en veremos, pero la incluyo igual.

## gnn_model_antiguo.py


---



In [ ]:
import tensorflow as tf
import tensorflow_gnn as tfgnn
import pickle

def create_graph_spec(n_robots):
  return tfgnn.GraphTensorSpec.from_piece_specs(
      context_spec=tfgnn.ContextSpec.from_field_specs(features_spec={
                    'context': tf.TensorSpec(shape=(1,7), dtype=tf.int32)
      }),
      node_sets_spec={
          'dron':
              tfgnn.NodeSetSpec.from_field_specs(
                  features_spec={
                      'bateria':
                          tf.TensorSpec((None, 1), tf.int32),
                      'n_objetivos_cubiertos':
                          tf.TensorSpec((None, 1), tf.int32),
                      'n_objetivos_FOV':
                          tf.TensorSpec((None, 1), tf.int32),
                      'vector_atributos':
                          #tf.TensorSpec((None, 13), tf.int32)
                          tf.TensorSpec((None, 7), tf.int32)
                  },
                  sizes_spec=tf.TensorSpec((1,), tf.int32))
      },
      edge_sets_spec={
          'conexion':
              tfgnn.EdgeSetSpec.from_field_specs(
                  sizes_spec=tf.TensorSpec((1,), tf.int32),
                  adjacency_spec=tfgnn.AdjacencySpec.from_incident_node_sets(
                      'dron', 'dron'))
      })

def load_pickles(path='./data/data_', num_data_files=1):
  atributos_data = []
  adj_mat_data = []
  acciones_data = []
  for i in range(1,num_data_files+1):
    fil = open(path+str(i)+'.pkl', 'rb')
    data = pickle.load(fil)
    adj_mat_data.append(data[0])
    atributos_data.append(data[1])
    acciones_data.append(data[2])

  return atributos_data, adj_mat_data, acciones_data

def adjacent_list(adj_mat):
  adj_mat = adj_mat>0
  adj_mat = adj_mat.astype(int)
  adj_list = []
  for i in range(len(adj_mat)):
    for j in range(len(adj_mat)):
      if(i!=j and adj_mat[i,j]==1):
          adj_list.append([i,j])
  return np.array(adj_list)

def create_GraphTensor(atributos, adj_list, accion, for_eval=False):
  vecinos = len(atributos[:,0])
  if(for_eval):
    bateria = atributos[:,0].reshape(vecinos,1)
    n_obj_cubiertos = atributos[:,1].reshape(vecinos,1)
    n_obj_FOV = atributos[:,2].reshape(vecinos,1)
    Accion = tf.keras.utils.to_categorical([accion], num_classes=7)
  else:
    bateria = atributos[:,0]
    n_obj_cubiertos = atributos[:,1]
    n_obj_FOV = atributos[:,2]
    Accion = tf.keras.utils.to_categorical(accion, num_classes=7)

  dron = tfgnn.NodeSet.from_fields(sizes=[vecinos],
      features={
      'bateria': bateria.astype('int32'),
      'n_objetivos_cubiertos': n_obj_cubiertos.astype('int32'),
      'n_objetivos_FOV': n_obj_FOV.astype('int32'),
      'vector_atributos': atributos.astype('int32')})

  adj_drones = tfgnn.Adjacency.from_indices(source=('dron', adj_list[:,0].astype('int32')),
                                                target=('dron', adj_list[:,1].astype('int32')))

  conexion = tfgnn.EdgeSet.from_fields(sizes=[len(adj_list)], adjacency=adj_drones)

  context = tfgnn.Context.from_fields(features={'context': Accion.astype('int32')})

  return tfgnn.GraphTensor.from_pieces(node_sets={'dron': dron}, edge_sets={'conexion': conexion}, context=context)


def create_dataset(atributos, adj_list, acciones, n_robots, for_eval=False):
  ds = []
  atr = []
  mat = []
  acc = []
  lista_robots = np.arange(0, n_robots)
  for sim in range(0, len(atributos)):
    for step in range(0, len(atributos[sim])):
      if(len(adj_list[sim][step]) == 0):
        for robot in range(0, len(atributos[sim][step])):
          atr.append(np.array([atributos[sim][step][robot]]))
          mat.append(np.array([[0,0]]))
          acc.append(acciones[sim][step][robot])
      else:
        robots_no_conectados = np.delete(lista_robots, np.unique(adj_list[sim][step][:,0]))
        for robot in robots_no_conectados:
          atr.append(np.array([atributos[sim][step][robot]]))
          mat.append(np.array([[0,0]]))
          acc.append(acciones[sim][step][robot])
        for robot in np.unique(adj_list[sim][step][:,0]):
          mat.append(adj_list[sim][step][np.where(adj_list[sim][step][:,0] == robot)])
          atr.append(atributos[sim][step])
          acc.append(acciones[sim][step][robot])

  for i in range(0, len(mat)):
    ds.append(create_GraphTensor(atr[i], mat[i], acc[i], for_eval))

  return ds

def tfrecord_decode_fn(record):
  graph_record = tfgnn.parse_single_example(graph_spec, record, validate=True)
  feat = graph_record.context.get_features_dict()
  accion = feat.pop('context')
  output_graph = graph_record.replace_features(context=feat)
  return output_graph, accion


# Modelo GNN

def create_GNN_Model(graph_spec, message_dim, num_message_passing):
    input_graph = tf.keras.layers.Input(type_spec=graph_spec)
    graph = input_graph.merge_batch_to_components()

    def set_initial_node_state(node_set, node_set_name):
      return {'bateria': tf.keras.layers.Dense(64, activation='relu')(node_set['bateria']),
              'n_objetivos_cubiertos': tf.keras.layers.Dense(64, activation='relu')(node_set['n_objetivos_cubiertos']),
              'n_objetivos_FOV': tf.keras.layers.Dense(64, activation='relu')(node_set['n_objetivos_FOV']),
              'vector_atributos': tf.keras.layers.Dense(64, activation='relu')(node_set['vector_atributos'])}
    graph = tfgnn.keras.layers.MapFeatures(node_sets_fn=set_initial_node_state)(graph)

    #message_passing

    regularizer_l2 = tf.keras.regularizers.l2(0.0005)
    dense = tf.keras.layers.Dense(message_dim, activation="relu", kernel_regularizer=regularizer_l2)

    for i in range(num_message_passing):
      graph = tfgnn.keras.layers.GraphUpdate(
          node_sets={"dron": tfgnn.keras.layers.NodeSetUpdate(
              {
                  "conexion": tfgnn.keras.layers.SimpleConv(
                      sender_node_feature="vector_atributos",
                      message_fn = dense,
                      reduce_type = "sum",
                      receiver_tag=tfgnn.SOURCE,
                      receiver_feature="vector_atributos"
                  )
              }
          ,tfgnn.keras.layers.NextStateFromConcat(dense), node_input_feature="vector_atributos")}
      )(graph)

    # generamos el vector de atributos de los arcos a partir del traspaso de información de los nodos vecinos a estos
    new_embedding_feat = tfgnn.broadcast_node_to_edges(graph, "conexion", tfgnn.TARGET, feature_name="vector_atributos")
    feat = graph.edge_sets['conexion'].get_features_dict()
    feat['vector_atributos'] = new_embedding_feat
    graph = graph.replace_features(edge_sets={'conexion': feat})

    pooled_features = tfgnn.keras.layers.Pool(
        tfgnn.CONTEXT, "sum", edge_set_name="conexion", feature_name='vector_atributos')(graph)

    d1 = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizer_l2)(pooled_features)
    d2 = tf.keras.layers.Dropout(0.1)(d1)
    output = tf.keras.layers.Dense(7, activation='softmax')(d2)

    return tf.keras.Model(input_graph, output)


El siguiente es la versión definitiva del conjunto de funciones para la gestión del entorno de trabajo del modelo basado en GNN

## gnn_model.py

In [ ]:
import tensorflow as tf
import tensorflow_gnn as tfgnn
import pickle
import numpy as np

def create_graph_spec(n_robots):
  return tfgnn.GraphTensorSpec.from_piece_specs(
      context_spec=tfgnn.ContextSpec.from_field_specs(features_spec={
                    'context': tf.TensorSpec(shape=(1,7), dtype=tf.int32)
      }),
      node_sets_spec={
          'dron':
              tfgnn.NodeSetSpec.from_field_specs(
                  features_spec={
                      'vector_atributos':
                          #tf.TensorSpec((None, 13), tf.int32)
                          tf.TensorSpec((None, 7), tf.int32)
                  },
                  sizes_spec=tf.TensorSpec((1,), tf.int32))
      },
      edge_sets_spec={
          'conexion':
              tfgnn.EdgeSetSpec.from_field_specs(
                  sizes_spec=tf.TensorSpec((1,), tf.int32),
                  adjacency_spec=tfgnn.AdjacencySpec.from_incident_node_sets(
                      'dron', 'dron'))
      })

def load_pickles(path='./data/data_', num_data_files=1):
  atributos_data = []
  adj_mat_data = []
  acciones_data = []
  for i in range(1,num_data_files+1):
    fil = open(path+str(i)+'.pkl', 'rb')
    data = pickle.load(fil)
    adj_mat_data.append(data[0])
    atributos_data.append(data[1])
    acciones_data.append(data[2])

  return atributos_data, adj_mat_data, acciones_data

def adjacent_list(adj_mat):
  adj_mat = adj_mat>0
  adj_mat = adj_mat.astype(int)
  adj_list = []
  for i in range(len(adj_mat)):
    for j in range(len(adj_mat)):
      if(i!=j and adj_mat[i,j]==1):
          adj_list.append([i,j])
  return np.array(adj_list)

def create_GraphTensor(atributos, adj_list, accion, for_eval=False):
  vecinos = len(atributos[:,0])
  if(for_eval):
    Accion = tf.keras.utils.to_categorical([accion], num_classes=7)
  else:
    Accion = tf.keras.utils.to_categorical(accion, num_classes=7)

  dron = tfgnn.NodeSet.from_fields(sizes=[vecinos],
      features={'vector_atributos': atributos.astype('int32')})

  adj_drones = tfgnn.Adjacency.from_indices(source=('dron', np.zeros(len(adj_list)).astype('int32')),
                                                target=('dron', np.arange(1, len(adj_list)+1).astype('int32')))

  conexion = tfgnn.EdgeSet.from_fields(sizes=[len(adj_list)], adjacency=adj_drones)

  context = tfgnn.Context.from_fields(features={'context': Accion.astype('int32')})

  return tfgnn.GraphTensor.from_pieces(node_sets={'dron': dron}, edge_sets={'conexion': conexion}, context=context)


def create_dataset(atributos, adj_list, acciones, n_robots, for_eval=False):
  ds = []
  atr = []
  mat = []
  acc = []
  lista_robots = np.arange(0, n_robots)
  for sim in range(0, len(atributos)):
    for step in range(0, len(atributos[sim])):
      if(len(adj_list[sim][step]) == 0):
        for robot in range(0, len(atributos[sim][step])):
          atr.append(np.array([atributos[sim][step][robot]]))
          mat.append(np.array([]))
          acc.append(acciones[sim][step][robot])
      else:
        robots_no_conectados = np.delete(lista_robots, np.unique(adj_list[sim][step][:,0]))
        for robot in robots_no_conectados:
          atr.append(np.array([atributos[sim][step][robot]]))
          mat.append(np.array([]))
          acc.append(acciones[sim][step][robot])
        for robot in np.unique(adj_list[sim][step][:,0]):
          mat.append(adj_list[sim][step][np.where(adj_list[sim][step][:,0] == robot)[0]])
          atr.append(atributos[sim][step][np.unique(adj_list[sim][step])])
          acc.append(acciones[sim][step][robot])

  for i in range(0, len(mat)):
    ds.append(create_GraphTensor(atr[i], mat[i], acc[i], for_eval))

  return ds

def tfrecord_decode_fn(record):
  graph_record = tfgnn.parse_single_example(graph_spec, record, validate=True)
  feat = graph_record.context.get_features_dict()
  accion = feat.pop('context')[0]
  output_graph = graph_record.replace_features(context=feat)
  return output_graph, accion


# Modelo GNN

def create_GNN_Model(graph_spec, message_dim, num_message_passing):
    input_graph = tf.keras.layers.Input(type_spec=graph_spec)
    graph = input_graph.merge_batch_to_components()

    def set_initial_node_state(node_set, node_set_name):
      return {'vector_atributos': tf.keras.Sequential([
              tf.keras.layers.Dense(32, activation='relu'),
              tf.keras.layers.Dense(64, activation='relu'),
              tf.keras.layers.Dense(128, activation='relu'),
              tf.keras.layers.Dense(256, activation='relu'),
              tf.keras.layers.Dense(128, activation='relu')])(node_set['vector_atributos'])}

    graph = tfgnn.keras.layers.MapFeatures(node_sets_fn=set_initial_node_state)(graph)

    def dense_SOURCE():
      return tf.keras.Sequential([tf.keras.layers.Dense(message_dim, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.0005)),
                                  tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))])
    def dense_TARGET():
      return tf.keras.Sequential([tf.keras.layers.Dense(message_dim, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.0005)),
                                  tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))])

    for i in range(num_message_passing):
      graph = tfgnn.keras.layers.GraphUpdate(
          node_sets={"dron": tfgnn.keras.layers.NodeSetUpdate(
              {
                  "conexion": tfgnn.keras.layers.SimpleConv(
                      sender_node_feature="vector_atributos",
                      message_fn = dense_TARGET(),
                      reduce_type = "sum",
                      receiver_tag=tfgnn.TARGET,
                      receiver_feature="vector_atributos"
                  )
              }
          ,tfgnn.keras.layers.NextStateFromConcat(dense_TARGET()), node_input_feature="vector_atributos")}
      )(graph)
      graph = tfgnn.keras.layers.GraphUpdate(
          node_sets={"dron": tfgnn.keras.layers.NodeSetUpdate(
              {#se elige el tipo de arco sobre el que se hará la conv, en este caso sobre el unico tipo que es conexion dron-dron
                  "conexion": tfgnn.keras.layers.SimpleConv(
                      sender_node_feature="vector_atributos",
                      message_fn = dense_SOURCE(),
                      reduce_type = "sum",
                      receiver_tag=tfgnn.SOURCE,
                      receiver_feature="vector_atributos"
                  )
              }
          ,tfgnn.keras.layers.NextStateFromConcat(dense_SOURCE()), node_input_feature="vector_atributos")}
      )(graph)

    # volvemos a gener el vector de atributos de los arcos a partir del traspaso de información de los nodos transformados en el SimpleConv a sus arcos vecinos
    # Necesario para el paso del Pool

    pooled_features = tfgnn.keras.layers.Pool(tfgnn.CONTEXT, "sum", node_set_name='dron', feature_name='vector_atributos')(graph)
    d1 = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))(pooled_features)
    d2 = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))(d1)
    output = tf.keras.layers.Dense(7, activation = "softmax")(d2)

    return tf.keras.Model([input_graph], output)



def create_GNN_Model2(graph_spec, message_dim, num_message_passing):
    input_graph = tf.keras.layers.Input(type_spec=graph_spec)
    graph = input_graph.merge_batch_to_components()

    def set_initial_node_state(node_set, node_set_name):
      return {'vector_atributos': tf.keras.Sequential([
              tf.keras.layers.Dense(32, activation='relu')])(node_set['vector_atributos'])}

    graph = tfgnn.keras.layers.MapFeatures(node_sets_fn=set_initial_node_state)(graph)

    def dense_SOURCE():
      return tf.keras.Sequential([tf.keras.layers.Dense(message_dim, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.0005))])
    def dense_TARGET():
      return tf.keras.Sequential([tf.keras.layers.Dense(message_dim, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.0005))])

    for i in range(num_message_passing):
      graph = tfgnn.keras.layers.GraphUpdate(
          node_sets={"dron": tfgnn.keras.layers.NodeSetUpdate(
              {
                  "conexion": tfgnn.keras.layers.SimpleConv(
                      sender_node_feature="vector_atributos",
                      message_fn = dense_TARGET(),
                      reduce_type = "sum",
                      receiver_tag=tfgnn.TARGET,
                      receiver_feature="vector_atributos"
                  )
              }
          ,tfgnn.keras.layers.NextStateFromConcat(dense_TARGET()), node_input_feature="vector_atributos")}
      )(graph)
      graph = tfgnn.keras.layers.GraphUpdate(
          node_sets={"dron": tfgnn.keras.layers.NodeSetUpdate(
              {#se elige el tipo de arco sobre el que se hará la conv, en este caso sobre el unico tipo que es conexion dron-dron
                  "conexion": tfgnn.keras.layers.SimpleConv(
                      sender_node_feature="vector_atributos",
                      message_fn = dense_SOURCE(),
                      reduce_type = "sum",
                      receiver_tag=tfgnn.SOURCE,
                      receiver_feature="vector_atributos"
                  )
              }
          ,tfgnn.keras.layers.NextStateFromConcat(dense_SOURCE()), node_input_feature="vector_atributos")}
      )(graph)

    # volvemos a gener el vector de atributos de los arcos a partir del traspaso de información de los nodos transformados en el SimpleConv a sus arcos vecinos
    # Necesario para el paso del Pool

    pooled_features = tfgnn.keras.layers.Pool(tfgnn.CONTEXT, "sum", node_set_name='dron', feature_name='vector_atributos')(graph)
    d1 = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))(pooled_features)
    d2 = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))(d1)
    output = tf.keras.layers.Dense(7, activation = "softmax")(d2)

    return tf.keras.Model([input_graph], output)

  The GNN core of the model does `num_message_passing` many updates of node
  states conditioned on their neighbors and the edges connecting to them.
  More precisely:
  - Each edge computes a message by applying a dense layer `message_fn`
     to the concatenation of node states of both endpoints (by default)
      and the edge's own unchanging feature embedding.
  - Messages are summed up at the common TARGET nodes of edges.
  - At each node, a dense layer is applied to the concatenation of the old
      node state with the summed edge inputs to compute the new node state.
   Each iteration of the for-loop creates new Keras Layer objects, so each
   round of updates gets its own trainable variables.

## main.py

In [ ]:
import tensorflow_gnn as tfgnn
# Estructura de un pickle [dict{0:adj_mat, 1:atributos, 2:acciones}, step_de_simulacion]
import pickle
#lectura de los pickles y preparacion del dataset
# en funcion de un solo dron
drones = 5
graph_spec = create_graph_spec(drones)

#cargamos los pickles
atributos, adj_mat, acciones = load_pickles(path='data/Voraz/data_', num_data_files=100)
adj_list = [[] for i in range(len(adj_mat))]

# convertimos todas las adj_mat a listas de adjacencia (adj_list)
for simulacion, mat in enumerate(adj_mat):
  for paso_simulacion in range(0, len(adj_mat[simulacion])):
    adj_list[simulacion].append(adjacent_list(mat[paso_simulacion]))

dataset = create_dataset(atributos, adj_list, acciones, drones)

# realizar la grabación de TFRecords con lo que retornará esta función (la lista dataset generada arriba)
#filename = f'train_data/train_data_ddpg.tfrecord'
filename = f'train_data/train_data_voraz.tfrecord'
with tf.io.TFRecordWriter(filename) as writer:
  for graph_tensor in dataset:
    tf_example = tfgnn.write_example(graph_tensor)
    writer.write(tf_example.SerializeToString())

#tf_dataset = tf.data.TFRecordDataset(['train_data/train_data_ddpg.tfrecord']).map(tfrecord_decode_fn)
tf_dataset = tf.data.TFRecordDataset(['train_data/train_data_voraz.tfrecord']).map(tfrecord_decode_fn)
input_graph_spec, accion_spec = tf_dataset.element_spec


IMPORTANTE: cuando llamemos a la función de create_GNN_Model hay que en el tf.keras.Input(type_spec=)meter este graphspec
ya que es el que fue generado con la tfrecord_decode_fn y que se supone que se le ha quitado el atributo de la accion y así está en sintonia
con los grafos del train_ds (por eso se utiliza es el input_graph_spec)

In [ ]:
drones = 5
graph_spec = create_graph_spec(drones)
tf_dataset = tf.data.TFRecordDataset(['train_data/train_data_voraz.tfrecord']).map(tfrecord_decode_fn)

In [ ]:
# Esto sigue estando dentro del main.py (lo siguiente es experimentación de ayer, lo que sigue es entrenar esto, empezaremos con el modelo de la arquitectura v1)
#drones = 5
#graph_spec = create_graph_spec(drones)
#tf_dataset = tf.data.TFRecordDataset(['train_data/train_data_voraz.tfrecord']).map(tfrecord_decode_fn)
input_graph_spec, accion_spec = tf_dataset.element_spec
GNN_Model = create_GNN_Model(input_graph_spec, 128, 1)

In [ ]:
# Como tenemos muchos datos, podemos dividir en train, test y val para mejorar el aprendizaje y evitar sobreajustes
tf_dataset = tf_dataset.shuffle(64)
size = len(list(tf_dataset))
train_size = int(0.6 * size)
val_size = int(0.2 * size)
test_size = int(0.2 * size)

# Dividir los datos
train_dataset = tf_dataset.take(train_size)
test_dataset = tf_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

#crear batchs
train_dataset = train_dataset.batch(64)
test_dataset = test_dataset.batch(64)
val_dataset = val_dataset.batch(64)

In [ ]:
print(GNN_Model.summary())
tf.keras.utils.plot_model(GNN_Model, show_shapes=True)

In [ ]:
#Entrenamiento Normal
loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizador = tf.keras.optimizers.Adam(0.001)
GNN_Model.compile(optimizador, loss=loss_func, metrics=['accuracy'])

In [ ]:
#Entrenamiento con Cosine learning decay
epochs = 1000
total_steps = 235
initial_learning_rate = 0.002
final_learning_rate = 0.0001
lr_scheduler = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate, epochs*total_steps, warmup_target=final_learning_rate)
loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizador = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
GNN_Model.compile(optimizador, loss=loss_func, metrics=['accuracy'])

In [ ]:
history = GNN_Model.fit(train_dataset, epochs=1000, validation_data=val_dataset)

for k, hist in history.history.items():
  plt.plot(hist)
  plt.title(k)
  plt.show()

In [ ]:
for k, hist in history.history.items():
  plt.plot(hist)
  plt.title(k)
  plt.show()

In [ ]:
#Evaluación con el conjunto de test
GNN_Model = create_GNN_Model(input_graph_spec, 128, 1)
GNN_Model.load_weights("./parametros/GNN/GNN_Voraz_Model.h5")
resultados = GNN_Model.evaluate(test_dataset)
print("Resultados: ", resultados)

In [ ]:
example_graph = next(iter(train_dataset.skip(10)))
print(example_graph[1])
GNN_Model(example_graph[0])

In [ ]:
#Guardamos los pesos
GNN_Model.save_weights("./parametros/GNN/GNN_DDPG_Model.h5")

# Evaluador de los modelos
En esta sección se planteará el evaluador que se encargará de simular un mismo escenario para ambos modelos el centralizado y el descentralizad (actor del DDPG y el GNN_Model) lo que se hará será plantear un objeto entorno donde las variables aleatorias (aquellas que se les da valor usando random) sea predeterminadas, se correrán varias simulaciones de estas y se comparará la recompensa total obtenida. Las variables aleatorias del Entorno son la carga y descarga de la batería, donde aparecen los objetivos y el movimiento de estos
También se aprovechará la animacion que genera la clase Entorno para comparar el comportamiento de los drones respecto a los objetivos en ambos modelos.

## Estado_Determinista.py

In [ ]:
import numpy as np
from random import randrange
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython import display

# Funciones de comportamiento de la bateria

def descarga_bateria(estado_actual, semilla_step):
    if(semilla_step>0.85 and estado_actual>0):
        return estado_actual-1
    else:
        return estado_actual

def carga_bateria(estado_actual, semilla_step):
    if(semilla_step>0.05 and estado_actual<10):
        return estado_actual+1
    else:
        return estado_actual

#dim dimensión nxn de la grid, las estaciónes de carga estarán en las posiciónes (0,0) y (15,15)

def estado_inicio(dim, n_robots, semilla_pos_robots, n_obj, semilla_pos_obj):
    estado = {}
    pos_robots = []
    for i in range(0, n_robots):
        pos_robots.append(semilla_pos_robots[i])
    pos_robots = np.array(pos_robots, dtype='float32')
    #bateria = np.ones(n_robots)*100
    bateria = np.ones(n_robots)*10
    pos_obj = []

    for i in range(0, n_obj):
        pos_obj.append(semilla_pos_obj[i])
    pos_obj = np.array(pos_obj, dtype='float32')

    estado['pos_robots'] = pos_robots
    estado['bateria'] = bateria
    estado['pos_obj'] = pos_obj

    return estado


class Estado_Determinista:

    # constructor
    def __init__(self, pos_robots, bateria, pos_obj, semilla_mov_obj, semilla_baterias):
        self.pos_robots = pos_robots
        self.bateria = bateria
        self.pos_obj = pos_obj
        self.registro = []
        self.it = 0
        self.fov_range = 4
        self.obj_ya_cubierto = np.ones(len(pos_robots))*-1
        self.semilla_mov_obj = semilla_mov_obj
        self.semilla_baterias = semilla_baterias

    def reset_env(self, pos_robots, bateria, pos_obj):
        self.pos_robots = pos_robots
        self.bateria = bateria
        self.pos_obj = pos_obj

    def get_estado(self):
        estado = {}
        estado = (map(tuple,self.pos_robots.tolist()), tuple(self.bateria), map(tuple, self.pos_obj.tolist()), tuple(self.planificador()))
        estado = tuple(map(tuple, estado))

        return estado

    def get_estado_array(self):
        #return np.array([np.concatenate([self.pos_robots.flatten(), self.bateria.flatten(), self.pos_obj.flatten(), self.planificador().flatten()])])
        # claro porque el no sabe donde estan los obj, se guia por la info que obtiene del FOV de cada robot (usando el planificador)
        return np.array([np.concatenate([self.pos_robots.flatten(), self.bateria.flatten(), self.planificador().flatten()])])

    def get_estado_experto(self):
        return [self.pos_robots, self.bateria.flatten(), self.planificador().flatten(), self.obj_ya_cubierto]


    def planificador(self):
      #planificacion = np.random.randint(0,6, size=len(self.pos_robots))
      planificacion = np.ones(len(self.pos_robots))*40 # un numero alejado de sus opciones lo interpretarán como fuera del alcance de cualquier objetivo
      direcciones = {
          (0,1): 0,
          (1,1): 1,
          (1,-1): 2,
          (0,-1): 3,
          (-1,-1): 4,
          (-1,1): 5,
          (1,0): 2,
          (-1,0): 4
      }
      for i in range(0, len(self.pos_robots)):
        for j in range(0, len(self.pos_obj)):
            if(math.sqrt((self.pos_robots[i,0]-self.pos_obj[j,0])**2 + (self.pos_robots[i,1]-self.pos_obj[j,1])**2)<=self.fov_range):
              dir = (self.pos_obj[j] - self.pos_robots[i])/abs(self.pos_obj[j] - self.pos_robots[i])
              x = np.nan_to_num(dir)
              if(tuple(x) in direcciones):
                planificacion[i] = direcciones[tuple(x)]
      return planificacion


    def verif_accion(self, robot, accion):
        nueva_pos = None
        acciones_contrarias = {0:3, 1:4, 2:5, 3:0, 4:1, 5:2}
        acciones_esquinas = {(0, 0): 1, (30, 0): 5, (30, 30): 4, (0, 30): 2}
        if(accion==0):
            nueva_pos = [self.pos_robots[robot,0], self.pos_robots[robot,1]+1]
        elif(accion==1):
            nueva_pos = [self.pos_robots[robot,0]+1, self.pos_robots[robot,1]+0.5]
        elif(accion==2):
            nueva_pos = [self.pos_robots[robot,0]+1, self.pos_robots[robot,1]-0.5]
        elif(accion==3):
            nueva_pos = [self.pos_robots[robot,0], self.pos_robots[robot,1]-1]
        elif(accion==4):
            nueva_pos = [self.pos_robots[robot,0]-1, self.pos_robots[robot,1]-0.5]
        elif(accion==5):
            nueva_pos = [self.pos_robots[robot,0]-1, self.pos_robots[robot,1]+0.5]

        if(nueva_pos == None):
          return accion
        elif(tuple(nueva_pos) in acciones_esquinas):
          return acciones_esquinas[tuple(nueva_pos)]
        elif(nueva_pos[0]>30 or nueva_pos[0]<0 or nueva_pos[1]>30 or nueva_pos[1]<0):
          return acciones_contrarias[accion]
        else:
          return accion



    def step(self, accion, sim_step):
        # actualizar posicion de todos los i robots
        for i in range(0, len(self.pos_robots)):

            if(self.bateria[i]<=0):
                continue

            accion[i] = self.verif_accion(i, accion[i]) #correcion de acciones, en caso de que se intente salir del limite del mapa

            if(accion[i]==0):
                self.pos_robots[i] = [self.pos_robots[i,0], self.pos_robots[i,1]+1]
            elif(accion[i]==1):
                self.pos_robots[i] = [self.pos_robots[i,0]+1, self.pos_robots[i,1]+0.5]
            elif(accion[i]==2):
                self.pos_robots[i] = [self.pos_robots[i,0]+1, self.pos_robots[i,1]-0.5]
            elif(accion[i]==3):
                self.pos_robots[i] = [self.pos_robots[i,0], self.pos_robots[i,1]-1]
            elif(accion[i]==4):
                self.pos_robots[i] = [self.pos_robots[i,0]-1, self.pos_robots[i,1]-0.5]
            elif(accion[i]==5):
                self.pos_robots[i] = [self.pos_robots[i,0]-1, self.pos_robots[i,1]+0.5]
            else:
                pass # la accion 6 es que elige mantenerse en posicion

            # actualizar el estado de la bateria
            if((self.pos_robots[i]==[0,0]).all() or (self.pos_robots[i]==[15,15]).all()):
                self.bateria[i] = carga_bateria(self.bateria[i], self.semilla_baterias[sim_step])
            else:
                self.bateria[i] = descarga_bateria(self.bateria[i], self.semilla_baterias[sim_step])

            #actualizar pos objetivos
        for i in range(0, len(self.pos_obj)):
            dir = self.semilla_mov_obj[sim_step, i]
            if(dir==0 and self.pos_obj[i,0]<30 and self.pos_obj[i,1]+1<30):
                self.pos_obj[i] = [self.pos_obj[i,0], self.pos_obj[i,1]+1]
            elif(dir==1 and self.pos_obj[i,0]+1<30 and self.pos_obj[i,1]+0.5<30):
                self.pos_obj[i] = [self.pos_obj[i,0]+1, self.pos_obj[i,1]+0.5]
            elif(dir==2 and self.pos_obj[i,0]+1<30 and self.pos_obj[i,1]-0.5>=0):
                self.pos_obj[i] = [self.pos_obj[i,0]+1, self.pos_obj[i,1]-0.5]
            elif(dir==3 and self.pos_obj[i,0]<30 and self.pos_obj[i,1]-1>=0):
                self.pos_obj[i] = [self.pos_obj[i,0], self.pos_obj[i,1]-1]
            elif(dir==4 and self.pos_obj[i,0]-1>=0 and self.pos_obj[i,1]-0.5>=0):
                self.pos_obj[i] = [self.pos_obj[i,0]-1, self.pos_obj[i,1]-0.5]
            elif(dir==5 and self.pos_obj[i,0]-1>=0 and self.pos_obj[i,1]+0.5<30):
                self.pos_obj[i] = [self.pos_obj[i,0]-1, self.pos_obj[i,1]+0.5]
            else:
                pass
        self.insertar_registro()

        #actualizamos el vector obj_ya_cubierto
        for i in range(0, len(self.pos_robots)):
            if(self.bateria[i]<=0):
                self.obj_ya_cubierto[i] = -1
                continue
            FOV1 = {}
            for k in [0, 0.5, 1]:
                for l in [0, 0.5, 1]:
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]] = 0
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]] = 0

            self.obj_ya_cubierto[i] = -1 # primero empezamos diciendo que no está cubriendo a nadie, y después comprobamos

            for j in range(0, len(self.pos_obj)):
                if(tuple(self.pos_obj[j]) in FOV1):
                    self.obj_ya_cubierto[i] = j



    def evaluar_posiciones(self):
        #para cada robot, se va a evaluar su posición utilizando el siguiente proceso:
        # recorriendo la matriz donde está la posición de cada robot:
            # para cada posición de robot (osea, para cada robot) generar el FOV (uno de radio 1)
             # con el de radio 1 verificar y contar la cantidad de obj cuyas posiciónes estan en él
             # meter en un vector en cada posición para cada robot la siguiente operación: 100*n_obj_FOV_1 - penalizacion
        #Retornar el vector resultante (que serán las recompensas del sistema en el estado actual tras evaluarse). Esto servirá para la funcion R(s,a) ya que esta es
        # literal la recompensa por el estado en el que se está
        vector_recompensa = np.zeros(len(self.pos_robots))

        for i in range(0, len(self.pos_robots)):
            if(self.bateria[i]<=0):
                continue
            penalizacion = 0
            FOV1 = {}
            for k in [0, 0.5, 1]:
                for l in [0, 0.5, 1]:
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]] = 0
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]+l] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]] = 0
                    FOV1[self.pos_robots[i,0]-k, self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0]+k, self.pos_robots[i,1]-l] = 0
                    FOV1[self.pos_robots[i,0], self.pos_robots[i,1]] = 0

            self.obj_ya_cubierto[i] = -1 # primero empezamos diciendo que no está cubriendo a nadie, y después comprobamos

            for j in range(0, len(self.pos_obj)):
                if(tuple(self.pos_obj[j]) in FOV1 and j not in self.obj_ya_cubierto):
                    FOV1[tuple(self.pos_obj[j])] +=1
                    self.obj_ya_cubierto[i] = j

            if((self.pos_robots[i]==[0,0]).all() or (self.pos_robots[i]==[15,15]).all()):
              penalizacion = -20
            else:
              penalizacion = 0

            vector_recompensa[i] = 100*sum(FOV1.values()) + penalizacion
        return sum(vector_recompensa)

    def insertar_registro(self):
      self.registro.append([self.it , self.pos_robots[:,0].copy(), self.pos_robots[:,1].copy(), self.pos_obj[:,0].copy(), self.pos_obj[:,1].copy()])
      self.it = self.it + 1

    def obtener_registro(self):
      return self.registro

    def generar_animacion(self):
      #obtenemos las ubicaciones de los robots y los objetivos
      robots = []
      for it in range(0, self.it):
        robots.append(pd.DataFrame([it, self.obtener_registro()[it][1], self.obtener_registro()[it][2]]).transpose())
      df_robots = pd.concat(robots)
      df_robots.columns = ['it', 'x0', 'y0']

      objetivos = []
      for it in range(0, self.it):
        objetivos.append(pd.DataFrame([self.obtener_registro()[it][0], self.obtener_registro()[it][3], self.obtener_registro()[it][4]]).transpose())
      df_objetivos = pd.concat(objetivos)
      df_objetivos.columns = ['it', 'x0', 'y0']

      fig, ax = plt.subplots()
      ax.set_xlim([0, 31])
      ax.set_ylim([0, 31])

      def animate(i):
        p1 = ax.clear()
        p2 = ax.clear()
        ax.grid(True)
        ax.set_xticks(np.linspace(0, 30, 31))
        ax.set_yticks(np.linspace(0, 30, 31))
        df_img1 = df_robots[df_robots['it'] == i][['x0', 'y0']]
        df_img2 = df_objetivos[df_objetivos['it'] == i][['x0', 'y0']]
        ax.set_xlim([0, 31])
        ax.set_ylim([0, 31])
        p1 = plt.scatter(df_img1['x0'].squeeze(), df_img1['y0'].squeeze())
        p2 = plt.scatter(df_img2['x0'].squeeze(), df_img2['y0'].squeeze())

        return p1,p2

      ani = animation.FuncAnimation(fig, func = animate, frames = self.it, interval = 400, blit = True)
      ani.save("animacion.gif", writer="Pillow")


In [ ]:
"""Procesos de simulaciones para comparar sistemas en entornos deterministas"""
#las funciones de simulacion corren 1 simulación y retornan dos variables: la recompensa total y una serie temporal con las recompensas de cada step

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

SIMULACIONES = 100
DIMMENSION = 30 # nxn
n_robots = 5
n_obj = 6
steps = 50

def simulacion_DDPG(estado_inicial, DDPG_model, steps):
    recompensa_total=0
    df_recompensas = []
    df_estado_acciones = []

    estado = Estado_Determinista(estado_inicial['pos_robots'].copy(), estado_inicial['bateria'].copy(), estado_inicial['pos_obj'].copy(), semillas_mov_obj.copy(), semillas_baterias.copy())

    for j in range(steps):
        estado0 = estado.get_estado_array()
        accion = tf.squeeze(DDPG_model(estado0)).numpy()
        estado.step(np.round(accion), j)
        recompensa_total += estado.evaluar_posiciones()
        df_recompensas.append(estado.evaluar_posiciones())
        df_estado_acciones.append([estado0, accion])
    return recompensa_total, df_recompensas, df_estado_acciones

def simulacion_Voraz(estado_inicial, steps):
    recompensa_total = 0
    df_recompensas = []
    df_estado_acciones = []

    estado = Estado_Determinista(estado_inicial['pos_robots'].copy(), estado_inicial['bateria'].copy(), estado_inicial['pos_obj'].copy(), semillas_mov_obj.copy(), semillas_baterias.copy())

    for j in range(steps):
        estado0 =  estado.get_estado_experto()
        accion = algoritmo_voraz(estado0, n_robots)
        estado.step(accion, j)
        recompensa_total += estado.evaluar_posiciones()
        df_recompensas.append(estado.evaluar_posiciones())
        df_estado_acciones.append([estado0, accion])
    return recompensa_total, df_recompensas, df_estado_acciones

def simulacion_GNN_Model(estado_inicial, GNN_model, steps):
    recompensa_total=0
    df_recompensas = []
    df_estado_acciones = []

    estado = Estado_Determinista(estado_inicial['pos_robots'].copy(), estado_inicial['bateria'].copy(), estado_inicial['pos_obj'].copy(), semillas_mov_obj.copy(), semillas_baterias.copy())

    for j in range(steps):
        acciones_robots = np.zeros(len(estado_inicial['pos_robots']))
        estado0 = estado.get_estado()
        atributos, adj_mat, acciones = generar_datos(estado0, acciones_robots)
        adj_list = adjacent_list(adj_mat)
        robots_graphs = create_dataset([[atributos]], [[adj_list]], [[acciones]], len(acciones_robots), for_eval=True)
        for i, graph in enumerate(robots_graphs):
            acciones_robots[i] = np.argmax(tf.squeeze(GNN_model(graph)).numpy())
        estado.step(np.round(acciones_robots),j)
        recompensa_total += estado.evaluar_posiciones()
        df_recompensas.append(estado.evaluar_posiciones())
        df_estado_acciones.append([estado0, acciones_robots])
    return recompensa_total, df_recompensas, df_estado_acciones

df_recom_voraz = []
df_recom_ddpg = []
df_recom_gnn_voraz = []
df_recom_gnn_ddpg = []

df_estado_acciones_voraz = []
df_estado_acciones_ddpg = []
df_estado_acciones_gnn_voraz = []
df_estado_acciones_gnn_ddpg = []

for i in range(0, SIMULACIONES):
    print("Inicio simulación: ", i)
    semillas_inicio_obj = np.random.randint(1, 29, size=(n_obj,2))
    semillas_inicio_robots = np.random.randint(1, 29, size=(n_robots,2))

    semillas_baterias = np.random.rand(steps)
    semillas_mov_obj = np.random.randint(0,6, size=(steps, n_obj))

    estado_inicial = estado_inicio(DIMMENSION, n_robots, semillas_inicio_robots.copy(), n_obj, semillas_inicio_obj.copy())

    # Algoritmo Voraz
    print("------------ Algoritmo Voraz ----------------")
    recom_voraz = simulacion_Voraz(estado_inicial, steps)
    df_recom_voraz.append(recom_voraz[0])
    df_estado_acciones_voraz.append(recom_voraz[2])

    # Modelo DDPG
    print("------------ Modelo DDPG -------------------")
    DDPG_Model = create_actor_model(n_robots, n_obj)
    DDPG_Model.load_weights("./parametros/DDPG/actor_objetivo.h5")
    recom_ddpg = simulacion_DDPG(estado_inicial, DDPG_Model, steps)
    df_recom_ddpg.append(recom_ddpg[0])
    df_estado_acciones_ddpg.append(recom_ddpg[2])

    # Modelo GNN entrenado con Voraz
    print("------------- Modelo GNN con Voraz -----------------")
    GNN_Model = create_GNN_Model(input_graph_spec, 128, 1)
    GNN_Model.load_weights("./parametros/GNN/GNN_Voraz_Model.h5")
    recom_gnn_voraz = simulacion_GNN_Model(estado_inicial, GNN_Model, steps)
    df_recom_gnn_voraz.append(recom_gnn_voraz[0])
    df_estado_acciones_gnn_voraz.append(recom_gnn_voraz[2])

    # Modelo GNN entrenado con DDPG
    print("------------- Modelo GNN con DDPG -----------------")
    GNN_Model = create_GNN_Model(input_graph_spec, 128, 1)
    GNN_Model.load_weights("./parametros/GNN/GNN_DDPG_Model.h5")
    recom_gnn_ddpg = simulacion_GNN_Model(estado_inicial, GNN_Model, steps)
    df_recom_gnn_ddpg.append(recom_gnn_ddpg[0])
    df_estado_acciones_gnn_ddpg.append(recom_gnn_ddpg[2])

df_recom = [df_recom_voraz, df_recom_gnn_voraz, df_recom_ddpg, df_recom_gnn_ddpg]

#graficamos un Diagrama de cajas con ambas series de datos de las simulaciones para comparar
x_axis = ['Voraz', 'GNN-Voraz', 'DDPG', 'GNN-DDPG']
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14,4))

bxplot1 = ax.boxplot(
    df_recom,
    vert=True,
    patch_artist=True,
    labels=x_axis
)
ax.set_title('Resultados simulaciones')
ax.yaxis.grid(True)
ax.set_ylabel("Recompensas por simulación")
ax.set_yticks(np.array(range(0, int(max(df_recom[0])), 1000)))

plt.show()


In [ ]:
#Guardar en pickles los datos de la simulación (estructura del pickle: [elemento, simulació, {0:estado, 1:acciones}, step])
pickle.dump([df_estado_acciones_voraz, df_estado_acciones_ddpg, df_estado_acciones_gnn_voraz, df_estado_acciones_gnn_ddpg], open('datos_simulacion.pkl', 'wb'))

# Capa GNN deconstruida
Conujunto de pruebas realizadas para familiarizarse con el funcionamiento de la librería Tensorflow_GNN. En esta sección se pasará un grafo de ejemplo del dataset por la red definida y se observará como va cambiando la información a medida de que va pasando por cada una de las capas de la red. Esto incluye la merge_batch_to_component. 

In [ ]:
dataset[390].edge_sets['conexion'].adjacency.source

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 2])>

In [ ]:
dataset[390].node_sets['dron'].features

{'vector_atributos': <tf.Tensor: shape=(4, 7), dtype=int32, numpy=
array([[ 2,  0,  0,  5,  5,  0, 40],
       [ 3,  0,  0, 24,  7,  0, 40],
       [ 2,  0,  0,  5,  5,  0, 40],
       [ 1,  0,  0,  5,  5,  0, 40]])>}

In [ ]:
#example_graph = next(iter(train_ds))
example_graph = dataset[390]
new_embedding_feat = tfgnn.broadcast_node_to_edges(example_graph, "conexion", tfgnn.TARGET, feature_name="vector_atributos")
feat = example_graph.edge_sets['conexion'].get_features_dict()
feat['vector_atributos'] = new_embedding_feat
new_example_Graph = example_graph.replace_features(edge_sets={'conexion': feat})

In [ ]:
new_example_Graph.edge_sets['conexion'].features

{'vector_atributos': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[ 2,  0,  0,  5,  5,  0, 40],
       [ 1,  0,  0,  5,  5,  0, 40]])>}

**tfgnn.broadcast_node_to_edges** <br>
Given a particular edge set (identified by edge_set_name name), this operation collects node features from the specific incident node of each edge (as indicated by node_tag). For example, setting node_tag=tfgnn.SOURCE and reduce_type='sum' gathers the source node features over each edge

In [ ]:
new_embedding_feat

<tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[ 2,  0,  0,  5,  5,  0, 40],
       [ 1,  0,  0,  5,  5,  0, 40]])>

In [ ]:
example_graph.node_sets['dron'].features

{'vector_atributos': <tf.Tensor: shape=(4, 7), dtype=int32, numpy=
array([[ 2,  0,  0,  5,  5,  0, 40],
       [ 3,  0,  0, 24,  7,  0, 40],
       [ 2,  0,  0,  5,  5,  0, 40],
       [ 1,  0,  0,  5,  5,  0, 40]])>}

In [ ]:
#así extraemos del train_ds (Objeto Dataset de TF) el grafo que nos interesa que estabamos utilizando antes (el de la posicion 180)
train_dataset.skip(1).take(1)
for i in train_dataset.skip(25).take(1):
  example_train_graph = i[0]

In [ ]:
example_train_graph.edge_sets['conexion'].adjacency.source

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>

**merge_batch_to_components** <br>
Este metodo se utiliza para optimizar el procesado de un dataset cuando éste se ha dividido en lotes, la función convierte el conjunto de grafos del lote en un solo grafo. Cuando se llama a esta función en un GraphTensor por lotes, se fusionan todos los grafos del lote en un solo grafo. El GraphTensor resultante tiene una forma [] (es decir, es escalar) y sus características tienen la forma [total_num_items, *feature_shape] donde total_num_items es la suma de los elementos en todos los grafos del lote


In [ ]:
# Capa MapFeatures
def set_initial_node_state(node_set, node_set_name):
  return {'vector_atributos': tf.keras.layers.Dense(64, activation='relu')(node_set['vector_atributos'])}

res = tfgnn.keras.layers.MapFeatures(node_sets_fn=set_initial_node_state)
res1 = res(example_train_graph)
print(res1.node_sets['dron'].features) # output graph features
example_train_graph.node_sets['dron'].features # input graph features

{'vector_atributos': <tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[12.237822  ,  0.        ,  1.7812735 ,  0.        ,  8.389606  ,
        12.6832695 ,  0.        ,  0.        , 13.2269    ,  3.1431682 ,
         0.        ,  0.        ,  5.667432  ,  3.1080885 ,  0.        ,
         6.962925  ,  3.961216  ,  0.        ,  0.46432543,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , 12.890139  ,
         0.        ,  7.0868187 ,  2.1432338 ,  0.        ,  0.        ,
         0.        ,  3.2714636 ,  0.        ,  0.        , 15.277388  ,
         4.0187316 ,  2.140689  ,  3.753451  ,  0.        ,  0.        ,
         0.        ,  0.        , 11.741581  ,  0.        ,  4.900159  ,
         2.9012218 ,  0.        ,  0.        ,  3.1338434 ,  7.5561857 ,
         0.        , 10.334125  ,  6.001386  ,  0.        ,  4.1507072 ,
         0.        ,  4.266093  ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.    

{'vector_atributos': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[ 6,  0,  0, 27,  1,  0, 40]])>}

**tfgnn.keras.layers.MapFeatures** <br>
This layer transforms the feature maps of graph pieces (that is, EdgeSets, NodeSets, or the Context) by applying Keras Models to them. Those Models are built by user-supplied callbacks that receive a KerasTensor for the graph piece as input and return a dict of output features computed with the Keras functional API

**Conv Model** <br>
A continuación se muestra la parte de las capas de convolución que se realizan.
El principio de lo que se hace en la capa GNN definida es tomar los atributos de los nodos y realizar una convolución simple o *message_passing* a partir del vector de atributos inicializado por la función set_initial_node_state. Como queremos hacer un pooling final para obtener el vector de atributos a nivel del contexto pero la función Pool de tfgnn toma en cuenta nodos conectados y no conectados (que no nos interesan los no conectados), se ha hecho un truquto de hacer un *traspaso de información* pasando la información de los nodos a los arcos (especificamente de los nodos vecinos del nodo principal a su respectivo arco conector) y así aplicar la función pool de tfgnn a nivel de los arcos, estos vectores ya convolusionados se reducen/agregan mediante una fución de agregación cualquiera y se obtiene el vector de atributos a nivel del grafo entero

In [ ]:
def create_conv_Model(graph_spec, num_message_passing=4):
    input_graph = tf.keras.layers.Input(type_spec=graph_spec)
    graph = input_graph.merge_batch_to_components()


    def set_initial_node_state(node_set, node_set_name):
        return {'vector_atributos': tf.keras.layers.Dense(64, activation='relu')(node_set['vector_atributos'])}
    graph = tfgnn.keras.layers.MapFeatures(node_sets_fn=set_initial_node_state)(graph)


    regularizer_l2 = tf.keras.regularizers.l2(0.0005)
    dense = tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=regularizer_l2)

    #en el paso broadcast_node_to_edges es que se le da un vector de atributos a los arcos, antes de esto estan vacíos y todas las operaciones era solo de vectores atributos de los nodos

    new_embedding_feat = tfgnn.broadcast_node_to_edges(graph, "conexion", tfgnn.TARGET, feature_name="vector_atributos")
    feat = graph.edge_sets['conexion'].get_features_dict()
    feat['vector_atributos'] = new_embedding_feat
    graph = graph.replace_features(edge_sets={'conexion': feat})

    for i in range(num_message_passing):
      graph = tfgnn.keras.layers.GraphUpdate(
          node_sets={"dron": tfgnn.keras.layers.NodeSetUpdate(
              {
                  "conexion": tfgnn.keras.layers.SimpleConv(
                      sender_node_feature="vector_atributos",
                      message_fn = dense,
                      reduce_type = "sum",
                      receiver_tag=tfgnn.TARGET,
                      receiver_feature="vector_atributos"
                  )
              }
          ,tfgnn.keras.layers.NextStateFromConcat(dense), node_input_feature="vector_atributos")}
      )(graph)

    #en el paso broadcast_node_to_edges es que se le da un vector de atributos a los arcos, antes de esto estan vacíos y todas las operaciones era solo de vectores atributos de los nodos
    # Se vuelve a hacer por que es necesario para el paso del Pool
    new_embedding_feat = tfgnn.broadcast_node_to_edges(graph, "conexion", tfgnn.TARGET, feature_name="vector_atributos")
    feat = graph.edge_sets['conexion'].get_features_dict()
    feat['vector_atributos'] = new_embedding_feat
    graph = graph.replace_features(edge_sets={'conexion': feat})

    return tf.keras.Model(input_graph, graph)

conv_layer = create_conv_Model(input_graph_spec)

In [ ]:
print(dataset[390].node_sets['dron'].features['vector_atributos'], "Atributos dron")
print(dataset[390].edge_sets['conexion'].adjacency.target)
print("")
res = conv_layer(example_train_graph)
print(res)

tf.Tensor(
[[ 2  0  0  5  5  0 40]
 [ 3  0  0 24  7  0 40]
 [ 2  0  0  5  5  0 40]
 [ 1  0  0  5  5  0 40]], shape=(4, 7), dtype=int32) Atributos dron
tf.Tensor([0 3], shape=(2,), dtype=int32)

GraphTensor(
  context=Context(features={}, sizes=[1], shape=(), indices_dtype=tf.int32),
  node_set_names=['dron'],
  edge_set_names=['conexion'])


In [ ]:
res.node_sets['dron'].features['vector_atributos']
res.edge_sets['conexion'].features #son dos arcos que conectan el nodo principal con sus dos vecinos

{}

In [ ]:
# ahora pasamos ese GraphTensor res por la capa pool
pool_layer = tfgnn.keras.layers.Pool(
      tfgnn.CONTEXT, "sum", edge_set_name="conexion", feature_name='vector_atributos')
res = pool_layer(res)
res

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[19.009363  ,  0.        ,  0.        ,  0.        ,  0.        ,
        18.563732  ,  0.        , 20.771582  , 21.14408   , 21.843283  ,
        10.219215  , 25.545837  , 22.10379   ,  0.        ,  0.        ,
         0.        ,  3.4250286 , 10.482754  ,  0.        ,  0.        ,
        19.659416  , 20.61258   ,  0.        , 25.645147  ,  0.        ,
        25.8006    , 15.927441  , 14.479823  ,  0.        , 20.63419   ,
         0.        , 25.633198  ,  0.        ,  0.        ,  0.        ,
        29.221777  , 27.750946  ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , 21.2555    ,  0.        , 13.655063  ,
         0.6225467 ,  0.        , 27.801052  , 27.303993  ,  0.        ,
        23.779596  ,  6.2836914 ,  0.46319258,  9.299696  , 19.025944  ,
        17.762905  , 28.789902  , 22.660254  , 25.692825  ,  0.        ,
        23.37738   , 38.961933  ,  0.        ,  0.        ]],
      dtype=f

Y después el tensor de (1, 64) de arriba entrara en un clasificador  MLP

### Pruebas Parte 2

In [ ]:
# atributos[Simulación][Step de la Simulación][Vector de atributos del robot en el Step] proveniente del dataset
print(atributos[0][20])
# acciones[Simulación][Vector con todas las acciones de todos los robots en el Step][Accion del robot i]
print(acciones[0][20])
adj_list[0][2]

[[0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [2. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0.]
 [3. 0. 0. 0. 0. 0. 1.]]
[0. 0. 0. 2. 6.]


array([[0, 1],
       [0, 2],
       [0, 3],
       [0, 4],
       [1, 0],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 0],
       [2, 1],
       [2, 3],
       [2, 4],
       [3, 0],
       [3, 1],
       [3, 2],
       [3, 4],
       [4, 0],
       [4, 1],
       [4, 2],
       [4, 3]])

In [ ]:
atr = []
mat = []
acc = []
for sim in range(0, len(atributos)):
  for step in range(0, len(atributos[sim])):
    if(len(adj_list[sim][step]) == 0):
      for robot in range(0, len(atributos[sim][step])):
        atr.append(np.array([atributos[sim][step][robot]]))
        mat.append(np.array([[0,0]]))
        acc.append(acciones[sim][step][robot])
    else:
      for robot in np.unique(adj_list[sim][step][:,0]):
        mat.append(adj_list[sim][step][np.where(adj_list[sim][step][:,0] == robot)])
        atr.append(atributos[sim][step][np.unique(mat[-1])])
        acc.append(acciones[sim][step][robot])

In [ ]:
atr[0]

array([[3., 0., 0., 1., 1., 1., 1.],
       [3., 0., 0., 1., 1., 1., 1.],
       [3., 0., 0., 1., 1., 1., 1.],
       [3., 0., 0., 1., 1., 1., 1.],
       [3., 0., 0., 1., 1., 1., 1.]])

In [ ]:
# Ejemplo de creacion de un GraphTensor (ejemplo de una instancia) para el dataset
dron = tfgnn.NodeSet.from_fields(sizes=[5],
    features={
    'bateria': [2,0,3,3,1],
    'n_objetivos_cubertos':[1,0,0,0,0],
    'n_objetivos_FOV': [2,0,1,3,1],
    'distancia_vecinos': [[1,0,7,4],[6,2,4,4],[1,0,4,5],[1,2,2,4],[3,0,1,2]]})

conexiones = tfgnn.EdgeSet.from_fields(sizes=[4],
      adjacency=tfgnn.Adjacency.from_indices(
          source = ('dron', [0,0,0,0]),
          target = ('dron', [1,2,3,4])
      ))

contexto = tfgnn.Context.from_fields(
    features={'accion': [1]}
)
contexto.features['accion']

grafo = tfgnn.GraphTensor.from_pieces(node_sets={'dron':dron}, edge_sets={'conexion':conexiones}, context=contexto)
grafo
grafo.spec # esto es lo que servira para tf.keras.Input(type_spec=grafo.spec)

GraphTensorSpec({'context': ContextSpec({'features': {'accion': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, None), 'node_sets': {'dron': NodeSetSpec({'features': {'bateria': TensorSpec(shape=(5,), dtype=tf.int32, name=None), 'n_objetivos_cubertos': TensorSpec(shape=(5,), dtype=tf.int32, name=None), 'n_objetivos_FOV': TensorSpec(shape=(5,), dtype=tf.int32, name=None), 'distancia_vecinos': TensorSpec(shape=(5, 4), dtype=tf.int32, name=None)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, None)}, 'edge_sets': {'conexion': EdgeSetSpec({'features': {}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None), 'adjacency': AdjacencySpec({'#index.0': TensorSpec(shape=(4,), dtype=tf.int32, name=None), '#index.1': TensorSpec(shape=(4,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, {'#index.0': 'dron', '#index.1': 'dron'})}, TensorShape([]), tf.in

In [ ]:
# Ejemplo de creacion de un GraphTensor (ejemplo de una instancia) para el dataset
dron = tfgnn.NodeSet.from_fields(sizes=[5],
    features={
    'bateria': [2,0,3,0,0],
    'n_objetivos_cubertos':[1,0,0,0,0],
    'n_objetivos_FOV': [2,0,1,0,0],
    'distancia_vecinos': [[1,0,7,4],[6,2,4,4],[1,0,4,5],[0,0,0,0],[0,0,0,0]]})

conexiones = tfgnn.EdgeSet.from_fields(sizes=[4],
      adjacency=tfgnn.Adjacency.from_indices(
          source = ('dron', [0,0,0,0]),
          target = ('dron', [1,2,0,0])
      ))

contexto = tfgnn.Context.from_fields(
    features={'accion': [1]}
)
contexto.features['accion']

grafo1 = tfgnn.GraphTensor.from_pieces(node_sets={'dron':dron}, edge_sets={'conexion':conexiones}, context=contexto)
grafo1
grafo1.spec # esto es lo que servira para tf.keras.Input(type_spec=grafo.spec)

GraphTensorSpec({'context': ContextSpec({'features': {'accion': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, None), 'node_sets': {'dron': NodeSetSpec({'features': {'bateria': TensorSpec(shape=(5,), dtype=tf.int32, name=None), 'n_objetivos_cubertos': TensorSpec(shape=(5,), dtype=tf.int32, name=None), 'n_objetivos_FOV': TensorSpec(shape=(5,), dtype=tf.int32, name=None), 'distancia_vecinos': TensorSpec(shape=(5, 4), dtype=tf.int32, name=None)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, None)}, 'edge_sets': {'conexion': EdgeSetSpec({'features': {}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None), 'adjacency': AdjacencySpec({'#index.0': TensorSpec(shape=(4,), dtype=tf.int32, name=None), '#index.1': TensorSpec(shape=(4,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, {'#index.0': 'dron', '#index.1': 'dron'})}, TensorShape([]), tf.in

In [ ]:
spec = grafo1.spec
spec.is_compatible_with(grafo) #verificamos su un graph_spec es compatible con un GraphTensor

True

In [ ]:
# ¿Puede existir en tensorflow_gnn un grafo que es solo un nodo?: Si, abajo el ejemplo
dron = tfgnn.NodeSet.from_fields(sizes=[1],
    features={
    'bateria': [2],
    'n_objetivos_cubertos':[1],
    'n_objetivos_FOV': [2],
    'distancia_vecinos': []})

conexiones = tfgnn.EdgeSet.from_fields(sizes=[0],
      adjacency=tfgnn.Adjacency.from_indices(
          source = ('dron', [0]),
          target = ('dron', [0])
      ))

contexto = tfgnn.Context.from_fields(
    features={'accion': [1]}
)
contexto.features['accion']

grafo = tfgnn.GraphTensor.from_pieces(node_sets={'dron':dron}, edge_sets={'conexion':conexiones}, context=contexto)
grafo

GraphTensor(
  context=Context(features={'accion': <tf.Tensor: shape=(1,), dtype=tf.int32>}, sizes=[1], shape=(), indices_dtype=tf.int32),
  node_set_names=['dron'],
  edge_set_names=['conexion'])

In [ ]:
grafo1 = tfgnn.random_graph_tensor(grafo.spec)
grafo1.node_sets['dron'].features

{'bateria': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([69, 17, 57, 13, 38], dtype=int32)>, 'n_objetivos_cubertos': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([29, 11, 61, 29, 67], dtype=int32)>, 'n_objetivos_FOV': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([20, 11, 68, 50, 21], dtype=int32)>, 'distancia_vecinos': <tf.Tensor: shape=(5, 4), dtype=int32, numpy=
array([[55, 41, 91, 27],
       [72, 92, 46, 80],
       [19, 42, 64, 89],
       [21, 86, 45, 36],
       [44, 45, 92, 58]], dtype=int32)>}

In [ ]:
# adj_list del dataset de simulaciones del agente centralizado experto
adj_list

[[array([[0, 1],
         [0, 2],
         [0, 3],
         [0, 4],
         [1, 0],
         [1, 2],
         [1, 3],
         [1, 4],
         [2, 0],
         [2, 1],
         [2, 3],
         [2, 4],
         [3, 0],
         [3, 1],
         [3, 2],
         [3, 4],
         [4, 0],
         [4, 1],
         [4, 2],
         [4, 3]]),
  array([[0, 1],
         [0, 2],
         [0, 3],
         [0, 4],
         [1, 0],
         [1, 2],
         [1, 3],
         [1, 4],
         [2, 0],
         [2, 1],
         [2, 3],
         [2, 4],
         [3, 0],
         [3, 1],
         [3, 2],
         [3, 4],
         [4, 0],
         [4, 1],
         [4, 2],
         [4, 3]]),
  array([[0, 1],
         [0, 2],
         [0, 3],
         [0, 4],
         [1, 0],
         [1, 2],
         [1, 3],
         [1, 4],
         [2, 0],
         [2, 1],
         [2, 3],
         [2, 4],
         [3, 0],
         [3, 1],
         [3, 2],
         [3, 4],
         [4, 0],
         [4, 1],
         [